# EDA notebook -- scratchbook to play & understand the data
### Feel free to mess around!

In [115]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
from pylab import rcParams
from statistics import stdev, median, mean
rcParams['figure.figsize'] = 10, 6
display(HTML("<style>.container { width:95% !important; }</style>"))

In [5]:
root = "data/anime-recommendations-database/"
root2 = "data/anime-data-score-staff-synopsis-and-genre/"
anime_data = pd.read_csv(root + "anime.csv")

FileNotFoundError: [Errno 2] File data/anime-recommendations-database/anime.csv does not exist: 'data/anime-recommendations-database/anime.csv'

In [ ]:
print (anime_data.shape)
anime_data.head(10)

In [ ]:
rating_data = pd.read_csv(root + "rating.csv")

In [ ]:
print (rating_data.shape)
rating_data.head(10)

In [ ]:
plt.plot(rating_data.rating.value_counts())

In [ ]:
# Seems like there are a ton of -1 ratings (where people didn't rate). May be worthwhile to delete all of them (>1.4M data) 
rating_data.rating.value_counts().plot(kind='bar')
plt.show()

In [ ]:
# lol, the anime with the most ratings is deathnote
rating_data.anime_id.value_counts()[:50].plot(kind='bar')
plt.show()

In [ ]:
top_50_frequently_rated = list(rating_data.anime_id.value_counts()[:25].keys())

In [ ]:
anime_data[anime_data['anime_id'].isin(top_50_frequently_rated)]

In [ ]:
new_df = pd.read_csv(root2 + "dataanime.csv")

In [ ]:
print (new_df.shape)
new_df.head()

In [ ]:
new_df = pd.read_csv("data/tidy_anime.csv")

In [ ]:
print(new_df.shape)
new_df.head()

In [ ]:
new_df.columns.tolist()

In [ ]:
desired_cols = ['animeID', 'title_english', 'type', 'source', 'producers', 'genre', 'studio',
               'episodes', 'premiered', 'rating', 'score', 'scored_by', 'rank', 'popularity',
               'members', 'favorites', 'synopsis', 'background']
new_df[desired_cols].head()

# Join the datasets, clean

* Rename rating column (duplicated)
* Join anime_data and rating_data on anime_id
* Remove all ratings of -1

In [6]:
anime_data = anime_data.rename(columns={"rating": "overall_rating"})
anime_data.head()

NameError: name 'anime_data' is not defined

In [ ]:
rating_data = rating_data.rename(columns={"rating": "user_rating"})
rating_data.head()

In [ ]:
merged_df = pd.merge(anime_data, rating_data, on='anime_id')
merged_df.head()

In [7]:
out_df = pd.read_csv('data/relevant_output_data.csv')
out_df = out_df.drop(columns=['Unnamed: 0'])
out_df.head()

,animeID,title_english,synopsis
0,1,Cowboy Bebop,"In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: The Movie,"Another day, another bounty—such is the life o..."
2,6,Trigun,"Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,Witches are individuals with special powers li...
4,8,Beet the Vandel Buster,It is the dark century and the people are suff...


In [2]:
gen_df = pd.read_csv('data/model_training_data.csv')
gen_df = gen_df.drop(columns=['Unnamed: 0'])
gen_df.head()

,animeID,title_english,episodes,score,scored_by,rank,popularity,members,favorites,genre_Action,...,synopsis_embedded_296,synopsis_embedded_297,synopsis_embedded_298,synopsis_embedded_299,synopsis_embedded_300,premiered_Spring,premiered_Summer,premiered_Fall,premiered_Winter,premiered_Year
0,1,Cowboy Bebop,26.0,8.81,405664,26,39,795733,43460,1,...,-0.015616,0.003619,-0.010637,0.043498,-0.007210,1.0,0.0,0.0,0.0,1998
1,5,Cowboy Bebop: The Movie,1.0,8.41,120243,164,449,197791,776,1,...,0.013802,0.005209,-0.006860,0.036505,-0.017165,0.0,0.0,0.0,0.0,0
2,6,Trigun,26.0,8.30,212537,255,146,408548,10432,1,...,-0.004276,0.010292,-0.036921,0.017824,0.005914,1.0,0.0,0.0,0.0,1998
3,7,Witch Hunter Robin,26.0,7.33,32837,2371,1171,79397,537,1,...,-0.008391,0.026849,-0.003479,0.048176,-0.002020,0.0,1.0,0.0,0.0,2002
4,8,Beet the Vandel Buster,52.0,7.03,4894,3544,3704,11708,14,0,...,-0.009359,-0.016920,-0.021534,0.047166,-0.012552,0.0,0.0,1.0,0.0,2004


In [3]:
def std_z(x, avg, std):
    return (x-avg)/std

def avg_std_z(xs, avg):
    sum = 0
    for x in xs:
        sum += (x-avg)**2
    return (sum/len(xs))**(1/2)

def avg(vals):
    if(len(vals) == 1):
        return vals[0]
    return sum(vals)/len(vals)

In [190]:
# usable overall sample stdev for features
fs = ['scored_by','popularity','episodes','rank', 'members', 
      'favorites', 'genre_Action', 'genre_Comedy', 'genre_Fantasy', 
      'genre_Mystery', 'studio_Madhouse', 'genre_Drama', 
      'genre_Fantasy', 'genre_Horror', 'genre_Romance', 'source_Manga',
      'source_Original']
f_stdv = {}
for f in fs:
    feat = list(gen_df[f])
    f_stdv[f] = stdev(feat)
print(f_stdv)

{'scored_by': 96648.63221428858, 'popularity': 2868.050739389625, 'episodes': 44.939364036423385, 'rank': 2736.869440698026, 'members': 164257.15112195478, 'favorites': 6211.037964762604, 'genre_Action': 0.48850123851270644, 'genre_Comedy': 0.4974470826236696, 'genre_Fantasy': 0.4325309112497383, 'genre_Mystery': 0.286256986048879, 'studio_Madhouse': 0.22800310333419382, 'genre_Drama': 0.4414897661801869, 'genre_Horror': 0.21816386583849504, 'genre_Romance': 0.41756893347821583, 'source_Manga': 0.48040963501750594, 'source_Original': 0.4232105983740449}


In [10]:
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan']
insyn_list = list(out_df[out_df['title_english'].isin(in_list)]['synopsis'])
for syn in insyn_list:
    print(syn[:300]+"...")
out_list = ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 'Death Note', 'One Punch Man']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(out_df[out_df['title_english'] == t]['synopsis'])[0])
for syn in outsyn_list:
    print('>> \t%s'%(syn[:300]+"..."))

Centuries ago, mankind was slaughtered to near extinction by monstrous humanoid creatures called titans, forcing humans to hide in fear behind enormous concentric walls. What makes these giants truly terrifying is that their taste for human flesh is not born out of hunger but what appears to be out ...
>> 	In the year 2022, virtual reality has progressed by leaps and bounds, and a massive online role-playing game called Sword Art Online (SAO) is launched. With the aid of "NerveGear" technology, players can control their avatars within the game using nothing but their own thoughts. Kazuto Kirigaya, nic...
>> 	Five years after winning the World Martial Arts tournament, Gokuu is now living a peaceful life with his wife and son. This changes, however, with the arrival of a mysterious enemy named Raditz who presents himself as Gokuu's long-lost brother. He reveals that Gokuu is a warrior from the once powerf...
>> 	One year has passed since the Black Rebellion, a failed uprising against the

In [116]:
# SCORED_BY
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan']
insyn_list = list(gen_df[gen_df['title_english'].isin(in_list)]['scored_by'])
for syn in insyn_list:
    print(syn)
out_list = ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 'Death Note', 'One Punch Man']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['scored_by'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['scored_by']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

1038161
>> 	1007035, 	-0.32205318675372124
>> 	402779, 	-6.574143735332292
>> 	591272, 	-4.6238522963176685
>> 	1107955, 	0.7221416216760658
>> 	806647, 	-2.3954193111257798
AVG ST.Z: 2.9275220302411054
AVG SQ ST.Z: 364104.0584099551
ABS MEAN DIFF: 255023.40000000002


In [117]:
# POPULARITY
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan']
insyn_list = list(gen_df[gen_df['title_english'].isin(in_list)]['popularity'])
for syn in insyn_list:
    print(syn)
out_list = ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 'Death Note', 'One Punch Man']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['popularity'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['popularity']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

2
>> 	3, 	0.00034866886637187556
>> 	71, 	0.024058151779659413
>> 	24, 	0.007670715060181262
>> 	1, 	-0.00034866886637187556
>> 	5, 	0.0010460065991156266
AVG ST.Z: 0.00669444223434001
AVG SQ ST.Z: 32.42221460665511
ABS MEAN DIFF: 18.8


In [13]:
# EPISODES
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan']
insyn_list = list(gen_df[gen_df['title_english'].isin(in_list)]['episodes'])
for syn in insyn_list:
    print(syn)
out_list = ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 'Death Note', 'One Punch Man']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['episodes'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['episodes']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

25.0
>> 	25.0, 	0.0
>> 	291.0, 	5.919086878586151
>> 	25.0, 	0.0
>> 	37.0, 	0.267026475725691
>> 	12.0, 	-0.28927868203616525
AVG ST.Z: 1.2950784072696013
AVG SQ ST.Z: 119.22164233057687


In [14]:
# RANK
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan']
insyn_list = list(gen_df[gen_df['title_english'].isin(in_list)]['rank'])
for syn in insyn_list:
    print(syn)
out_list = ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 'Death Note', 'One Punch Man']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['rank'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['rank']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

116
>> 	1440, 	0.4837643989558814
>> 	249, 	0.0485956684751754
>> 	17, 	-0.03617271563189748
>> 	54, 	-0.02265361989068327
>> 	46, 	-0.02557666761851337
AVG ST.Z: 0.12335261411443019
AVG SQ ST.Z: 598.1989635564408


In [15]:
# MEMBERS
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan']
insyn_list = list(gen_df[gen_df['title_english'].isin(in_list)]['members'])
for syn in insyn_list:
    print(syn)
out_list = ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 'Death Note', 'One Punch Man']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['members'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['members']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

1500958
>> 	1442099, 	-0.3583344749252311
>> 	591990, 	-5.533810819141294
>> 	869302, 	-3.8455312032717472
>> 	1610561, 	0.6672647081199155
>> 	1195384, 	-1.8603390958188648
AVG ST.Z: 2.4530560602554106
AVG SQ ST.Z: 516539.4072722816


In [16]:
# FAVORITES
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan']
insyn_list = list(gen_df[gen_df['title_english'].isin(in_list)]['favorites'])
for syn in insyn_list:
    print(syn)
out_list = ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 'Death Note', 'One Punch Man']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['favorites'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['favorites']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

70555
>> 	53268, 	-2.783270702590326
>> 	23468, 	-7.581180515582268
>> 	47637, 	-3.6898824528238032
>> 	96146, 	4.1202453028280805
>> 	35969, 	-5.568473449400648
AVG ST.Z: 4.748610484645026
AVG SQ ST.Z: 31280.509391632353


In [17]:
# from data_modeling EXAMPLE 1 (Cosine, normalized), ['Attack on Titan'] input,
# ['Attack on Titan Season 2', 'Fullmetal Alchemist: Brotherhood', 'Death Note',
# 'Code Geass: Lelouch of the Rebellion', 'Code Geass: Lelouch of the Rebellion R2'] output
in_list = ['Attack on Titan']
insyn_list = list(out_df[out_df['title_english'].isin(in_list)]['synopsis'])
for syn in insyn_list:
    print(syn[:300]+"...")
out_list = ['Attack on Titan Season 2', 'Fullmetal Alchemist: Brotherhood', 'Death Note',
            'Code Geass: Lelouch of the Rebellion', 'Code Geass: Lelouch of the Rebellion R2']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(out_df[out_df['title_english'] == t]['synopsis'])[0])
for syn in outsyn_list:
    print('>> \t%s'%(syn[:300]+"..."))

Centuries ago, mankind was slaughtered to near extinction by monstrous humanoid creatures called titans, forcing humans to hide in fear behind enormous concentric walls. What makes these giants truly terrifying is that their taste for human flesh is not born out of hunger but what appears to be out ...
>> 	For centuries, humanity has been hunted by giant, mysterious predators known as the Titans. Three mighty walls—Wall Maria, Rose, and Sheena—provided peace and protection for humanity for over a hundred years. That peace, however, was shattered when the Colossus Titan and Armored Titan appeared and d...
>> 	"In order for something to be obtained, something of equal value must be lost." Alchemy is bound by this Law of Equivalent Exchange—something the young brothers Edward and Alphonse Elric only realize after attempting human transmutation: the one forbidden act of alchemy. They pay a terrible price fo...
>> 	A shinigami, as a god of death, can kill any person—provided they see their 

In [18]:
# SCORED_BY
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan']
insyn_list = list(gen_df[gen_df['title_english'].isin(in_list)]['scored_by'])
for syn in insyn_list:
    print(syn)
out_list = ['Attack on Titan Season 2', 'Fullmetal Alchemist: Brotherhood', 'Death Note',
            'Code Geass: Lelouch of the Rebellion', 'Code Geass: Lelouch of the Rebellion R2']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['scored_by'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['scored_by']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

1038161
>> 	439965, 	-6.189389195634808
>> 	826899, 	-2.1858767699018395
>> 	1107955, 	0.7221416216760658
>> 	681621, 	-3.689033065770474
>> 	591272, 	-4.6238522963176685
AVG ST.Z: 3.4820585898601712
AVG SQ ST.Z: 383189.72831144626


In [19]:
# POPULARITY
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan']
insyn_list = list(gen_df[gen_df['title_english'].isin(in_list)]['popularity'])
for syn in insyn_list:
    print(syn)
out_list = ['Attack on Titan Season 2', 'Fullmetal Alchemist: Brotherhood', 'Death Note',
            'Code Geass: Lelouch of the Rebellion', 'Code Geass: Lelouch of the Rebellion R2']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['popularity'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['popularity']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

2
>> 	41, 	0.013598085788503146
>> 	4, 	0.0006973377327437511
>> 	1, 	-0.00034866886637187556
>> 	11, 	0.0031380197973468796
>> 	24, 	0.007670715060181262
AVG ST.Z: 0.005090565449029382
AVG SQ ST.Z: 20.449938875214272


In [20]:
# EPISODES
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan']
insyn_list = list(gen_df[gen_df['title_english'].isin(in_list)]['episodes'])
for syn in insyn_list:
    print(syn)
out_list = ['Attack on Titan Season 2', 'Fullmetal Alchemist: Brotherhood', 'Death Note',
            'Code Geass: Lelouch of the Rebellion', 'Code Geass: Lelouch of the Rebellion R2']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['episodes'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['episodes']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

25.0
>> 	12.0, 	-0.28927868203616525
>> 	64.0, 	0.8678360461084957
>> 	37.0, 	0.267026475725691
>> 	25.0, 	0.0
>> 	25.0, 	0.0
AVG ST.Z: 0.28482824077407043
AVG SQ ST.Z: 19.15202339179858


In [21]:
# RANK
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan']
insyn_list = list(gen_df[gen_df['title_english'].isin(in_list)]['rank'])
for syn in insyn_list:
    print(syn)
out_list = ['Attack on Titan Season 2', 'Fullmetal Alchemist: Brotherhood', 'Death Note',
            'Code Geass: Lelouch of the Rebellion', 'Code Geass: Lelouch of the Rebellion R2']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['rank'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['rank']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

116
>> 	143, 	0.009865286081426585
>> 	1, 	-0.04201881108755768
>> 	54, 	-0.02265361989068327
>> 	32, 	-0.03069200114221604
>> 	17, 	-0.03617271563189748
AVG ST.Z: 0.028280486766756208
AVG SQ ST.Z: 83.25262758616091


In [22]:
# MEMBERS
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan']
insyn_list = list(gen_df[gen_df['title_english'].isin(in_list)]['members'])
for syn in insyn_list:
    print(syn)
out_list = ['Attack on Titan Season 2', 'Fullmetal Alchemist: Brotherhood', 'Death Note',
            'Code Geass: Lelouch of the Rebellion', 'Code Geass: Lelouch of the Rebellion R2']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['members'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['members']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

1500958
>> 	781129, 	-4.382329749927015
>> 	1355349, 	-0.8864697762345262
>> 	1610561, 	0.6672647081199155
>> 	1082349, 	-2.548497871421126
>> 	869302, 	-3.8455312032717472
AVG ST.Z: 2.466018661794866
AVG SQ ST.Z: 474466.3075389021


In [23]:
# FAVORITES
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan']
insyn_list = list(gen_df[gen_df['title_english'].isin(in_list)]['favorites'])
for syn in insyn_list:
    print(syn)
out_list = ['Attack on Titan Season 2', 'Fullmetal Alchemist: Brotherhood', 'Death Note',
            'Code Geass: Lelouch of the Rebellion', 'Code Geass: Lelouch of the Rebellion R2']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['favorites'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['favorites']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

70555
>> 	8670, 	-9.963713046208266
>> 	120331, 	8.014119424546541
>> 	96146, 	4.1202453028280805
>> 	68684, 	-0.3012378946344941
>> 	47637, 	-3.6898824528238032
AVG ST.Z: 5.217839624208237
AVG SQ ST.Z: 38706.68688224297


In [24]:
# from data_modeling EXAMPLE 2 (Cosine, un-normalized)
# ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
#  'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
#  'Attack on Titan Season 3'] input,
# ['anohana: The Flower We Saw That Day', 'Puella Magi Madoka Magica the Movie: Rebellion',
#  'Kuroko\'s Basketball', 'Vampire Knight', 'Maid Sama!'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = list(out_df[out_df['title_english'].isin(in_list)]['synopsis'])
for syn in insyn_list:
    print(syn[:300]+"...")
out_list = ['anohana: The Flower We Saw That Day', 'Puella Magi Madoka Magica the Movie: Rebellion', 
            'Kuroko\'s Basketball', 'Vampire Knight', 'Maid Sama!']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(out_df[out_df['title_english'] == t]['synopsis'])[0])
for syn in outsyn_list:
    print('>> \t%s'%(syn[:300]+"..."))

Centuries ago, mankind was slaughtered to near extinction by monstrous humanoid creatures called titans, forcing humans to hide in fear behind enormous concentric walls. What makes these giants truly terrifying is that their taste for human flesh is not born out of hunger but what appears to be out ...
Recap of the events in Shingeki no Kyojin that happened prior to episode 14....
Recap of episodes 1-13....
Recap of episodes 14-25....
For centuries, humanity has been hunted by giant, mysterious predators known as the Titans. Three mighty walls—Wall Maria, Rose, and Sheena—provided peace and protection for humanity for over a hundred years. That peace, however, was shattered when the Colossus Titan and Armored Titan appeared and d...
On his first day of junior high, Eren Yeager comes face-to-face with a titan—and has his lunch stolen! From that day on, he holds a grudge against titans for taking his favorite food from him, a cheeseburger, vowing to eliminate their kind once and for all.

In [208]:
# SCORED_BY
# from data_modeling EXAMPLE 2 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['scored_by'])[0])
print(insyn_list)
out_list = ['anohana: The Flower We Saw That Day', 'Puella Magi Madoka Magica the Movie: Rebellion', 
            'Kuroko\'s Basketball', 'Vampire Knight', 'Maid Sama!']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['scored_by'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['scored_by']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[1038161, 36288, 20514, 15438, 439965, 43670, 154179]
>> 	439345, 	1.9617453000226677
>> 	123238, 	-1.3089373031116436
>> 	315061, 	0.6758088397483152
>> 	220644, 	-0.30110100198290957
>> 	339932, 	0.9331430557655291
AVG ST.Z: 1.0361471001262132
AVG SQ ST.Z: 114191.87718485059
ABS MEAN DIFF: 37899


In [209]:
# POPULARITY
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['popularity'])[0])
print(insyn_list)
out_list = ['anohana: The Flower We Saw That Day', 'Puella Magi Madoka Magica the Movie: Rebellion', 
            'Kuroko\'s Basketball', 'Vampire Knight', 'Maid Sama!']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['popularity'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['popularity']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[2, 1349, 1608, 1714, 41, 1050, 159]
>> 	49, 	-0.2779388963364379
>> 	416, 	-0.1499774223779596
>> 	88, 	-0.2643408105479348
>> 	173, 	-0.23470395690632534
>> 	76, 	-0.26852483694439727
AVG ST.Z: 0.239097184622611
AVG SQ ST.Z: 698.7820161698846


In [210]:
# EPISODES
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['episodes'])[0])
print(insyn_list)
out_list = ['anohana: The Flower We Saw That Day', 'Puella Magi Madoka Magica the Movie: Rebellion', 
            'Kuroko\'s Basketball', 'Vampire Knight', 'Maid Sama!']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['episodes'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['episodes']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[25.0, 1.0, 1.0, 1.0, 12.0, 12.0, 12.0]
>> 	11.0, 	0.04132552600516647
>> 	1.0, 	-0.18119653709957603
>> 	25.0, 	0.352856414351806
>> 	13.0, 	0.08582993862611497
>> 	26.0, 	0.37510862066228023
AVG ST.Z: 0.20726340734898874
AVG SQ ST.Z: 11.137727757125182


In [211]:
# RANK
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['rank'])[0])
print(insyn_list)
out_list = ['anohana: The Flower We Saw That Day', 'Puella Magi Madoka Magica the Movie: Rebellion', 
            'Kuroko\'s Basketball', 'Vampire Knight', 'Maid Sama!']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['rank'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['rank']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[116, 3529, 1166, 1002, 143, 3454, 113]
>> 	93, 	-0.46309427573765427
>> 	131, 	-0.44920979903046127
>> 	212, 	-0.4196139407861815
>> 	2602, 	0.4536465679030606
>> 	336, 	-0.374306701004815
AVG ST.Z: 0.43197425689243446
AVG SQ ST.Z: 1185.5556916312214


In [212]:
# MEMBERS
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['members'])[0])
print(insyn_list)
out_list = ['anohana: The Flower We Saw That Day', 'Puella Magi Madoka Magica the Movie: Rebellion', 
            'Kuroko\'s Basketball', 'Vampire Knight', 'Maid Sama!']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['members'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['members']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[1500958, 68519, 52767, 48292, 781129, 89969, 394365]
>> 	734942, 	1.920851355776373
>> 	208482, 	-1.284245021483471
>> 	534177, 	0.6985910241641468
>> 	371097, 	-0.2942424621473211
>> 	579513, 	0.9745972722351347
AVG ST.Z: 1.0345054271612892
AVG SQ ST.Z: 192446.06575598032


In [213]:
# FAVORITES
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['favorites'])[0])
print(insyn_list)
out_list = ['anohana: The Flower We Saw That Day', 'Puella Magi Madoka Magica the Movie: Rebellion', 
            'Kuroko\'s Basketball', 'Vampire Knight', 'Maid Sama!']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['favorites'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['favorites']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[70555, 166, 97, 88, 8670, 407, 3662]
>> 	22969, 	1.7742146076441634
>> 	5275, 	-1.0745845947410524
>> 	16005, 	0.6529849453060463
>> 	8111, 	-0.6179781440818192
>> 	15410, 	0.5571877527312072
AVG ST.Z: 0.9353900089008576
AVG SQ ST.Z: 6467.425943818414


In [214]:
# FAVORITES
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Action'])[0])
print(insyn_list)
out_list = ['anohana: The Flower We Saw That Day', 'Puella Magi Madoka Magica the Movie: Rebellion', 
            'Kuroko\'s Basketball', 'Vampire Knight', 'Maid Sama!']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Action'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['genre_Action']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[1, 1, 1, 1, 1, 0, 1]
>> 	0, 	-1.7546380429914956
>> 	0, 	-1.7546380429914956
>> 	0, 	-1.7546380429914956
>> 	0, 	-1.7546380429914956
>> 	0, 	-1.7546380429914956
AVG ST.Z: 1.7546380429914954
AVG SQ ST.Z: 0.8571428571428571
ABS MEAN DIFF: 0.8571428571428571


In [215]:
# FAVORITES
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Comedy'])[0])
print(insyn_list)
out_list = ['anohana: The Flower We Saw That Day', 'Puella Magi Madoka Magica the Movie: Rebellion', 
            'Kuroko\'s Basketball', 'Vampire Knight', 'Maid Sama!']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Comedy'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['genre_Comedy']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[0, 0, 0, 0, 0, 1, 0]
>> 	0, 	-0.28718058231174237
>> 	0, 	-0.28718058231174237
>> 	1, 	1.7230834938704545
>> 	0, 	-0.28718058231174237
>> 	1, 	1.7230834938704545
AVG ST.Z: 0.8615417469352271
AVG SQ ST.Z: 0.5532833351724882
ABS MEAN DIFF: 0.2571428571428572


In [216]:
# FAVORITES
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Mystery'])[0])
print(insyn_list)
out_list = ['anohana: The Flower We Saw That Day', 'Puella Magi Madoka Magica the Movie: Rebellion', 
            'Kuroko\'s Basketball', 'Vampire Knight', 'Maid Sama!']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Mystery'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['genre_Mystery']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[1, 0, 0, 0, 1, 0, 1]
>> 	0, 	-1.4971562248554138
>> 	1, 	1.9962082998072184
>> 	0, 	-1.4971562248554138
>> 	1, 	1.9962082998072184
>> 	0, 	-1.4971562248554138
AVG ST.Z: 1.6967770548361358
AVG SQ ST.Z: 0.490730401066219
ABS MEAN DIFF: 0.028571428571428525


In [217]:
# FAVORITES
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['studio_Madhouse'])[0])
print(insyn_list)
out_list = ['anohana: The Flower We Saw That Day', 'Puella Magi Madoka Magica the Movie: Rebellion', 
            'Kuroko\'s Basketball', 'Vampire Knight', 'Maid Sama!']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['studio_Madhouse'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['studio_Madhouse']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[0, 0, 0, 0, 0, 0, 0]
>> 	0, 	0.0
>> 	0, 	0.0
>> 	0, 	0.0
>> 	0, 	0.0
>> 	0, 	0.0
AVG ST.Z: 0.0
AVG SQ ST.Z: 0.0
ABS MEAN DIFF: 0


In [31]:
# from data_modeling EXAMPLE 2 (Cosine, normalized)
# ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
#  'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
#  'Attack on Titan Season 3'] input,
# ['Gun Samurai Recap', 'Marches Comes in Like a Lion: Special Omnibus Episode',
#  'Berserk: Recollections of the Witch', 'So, I Can\'t Play H!',
#  'Tsukigakirei: First Half - The Road So Far'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = list(out_df[out_df['title_english'].isin(in_list)]['synopsis'])
for syn in insyn_list:
    print(syn[:300]+"...")
out_list = ['Gun Samurai Recap', 'Marches Comes in Like a Lion: Special Omnibus Episode',
            'Berserk: Recollections of the Witch', 'So, I Can\'t Play H!',
            'Tsukigakirei: First Half - The Road So Far']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(out_df[out_df['title_english'] == t]['synopsis'])[0])
for syn in outsyn_list:
    print('>> \t%s'%(syn[:300]+"..."))

Centuries ago, mankind was slaughtered to near extinction by monstrous humanoid creatures called titans, forcing humans to hide in fear behind enormous concentric walls. What makes these giants truly terrifying is that their taste for human flesh is not born out of hunger but what appears to be out ...
Recap of the events in Shingeki no Kyojin that happened prior to episode 14....
Recap of episodes 1-13....
Recap of episodes 14-25....
For centuries, humanity has been hunted by giant, mysterious predators known as the Titans. Three mighty walls—Wall Maria, Rose, and Sheena—provided peace and protection for humanity for over a hundred years. That peace, however, was shattered when the Colossus Titan and Armored Titan appeared and d...
On his first day of junior high, Eren Yeager comes face-to-face with a titan—and has his lunch stolen! From that day on, he holds a grudge against titans for taking his favorite food from him, a cheeseburger, vowing to eliminate their kind once and for all.

In [218]:
# SCORED_BY
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['scored_by'])[0])
print(insyn_list)
out_list = ['Gun Samurai Recap', 'Marches Comes in Like a Lion: Special Omnibus Episode',
            'Berserk: Recollections of the Witch', 'So, I Can\'t Play H!',
            'Tsukigakirei: First Half - The Road So Far']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['scored_by'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['scored_by']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[1038161, 36288, 20514, 15438, 439965, 43670, 154179]
>> 	45, 	-2.5835854505045366
>> 	4589, 	-2.536569782554626
>> 	4083, 	-2.541805242057851
>> 	133619, 	-1.2015276092322378
>> 	5019, 	-2.532120676652676
AVG ST.Z: 2.2791217522003855
AVG SQ ST.Z: 226352.5725641306


In [219]:
# POPULARITY
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['popularity'])[0])
print(insyn_list)
out_list = ['Gun Samurai Recap', 'Marches Comes in Like a Lion: Special Omnibus Episode',
            'Berserk: Recollections of the Witch', 'So, I Can\'t Play H!',
            'Tsukigakirei: First Half - The Road So Far']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['popularity'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['popularity']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[2, 1349, 1608, 1714, 41, 1050, 159]
>> 	11050, 	3.5577673026205647
>> 	4185, 	1.1641555349776394
>> 	4249, 	1.1864703424254395
>> 	320, 	-0.18344963354965965
>> 	4118, 	1.1407947209307236
AVG ST.Z: 1.4465275069008054
AVG SQ ST.Z: 5250.294845364907


In [220]:
# EPISODES
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['episodes'])[0])
print(insyn_list)
out_list = ['Gun Samurai Recap', 'Marches Comes in Like a Lion: Special Omnibus Episode',
            'Berserk: Recollections of the Witch', 'So, I Can\'t Play H!',
            'Tsukigakirei: First Half - The Road So Far']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['episodes'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['episodes']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[25.0, 1.0, 1.0, 1.0, 12.0, 12.0, 12.0]
>> 	1.0, 	-0.18119653709957603
>> 	1.0, 	-0.18119653709957603
>> 	1.0, 	-0.18119653709957603
>> 	12.0, 	0.06357773231564072
>> 	1.0, 	-0.18119653709957603
AVG ST.Z: 0.15767277614278896
AVG SQ ST.Z: 7.394427024483246


In [221]:
# RANK
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['rank'])[0])
print(insyn_list)
out_list = ['Gun Samurai Recap', 'Marches Comes in Like a Lion: Special Omnibus Episode',
            'Berserk: Recollections of the Witch', 'So, I Can\'t Play H!',
            'Tsukigakirei: First Half - The Road So Far']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['rank'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['rank']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[116, 3529, 1166, 1002, 143, 3454, 113]
>> 	11051, 	3.5407503494576242
>> 	4747, 	1.2373887399275059
>> 	8229, 	2.5096452634655564
>> 	4241, 	1.0525059711422522
>> 	5603, 	1.5501548468053266
AVG ST.Z: 1.978089034159653
AVG SQ ST.Z: 5980.809614158967


In [222]:
# MEMBERS
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['members'])[0])
print(insyn_list)
out_list = ['Gun Samurai Recap', 'Marches Comes in Like a Lion: Special Omnibus Episode',
            'Berserk: Recollections of the Witch', 'So, I Can\'t Play H!',
            'Tsukigakirei: First Half - The Road So Far']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['members'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['members']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[1500958, 68519, 52767, 48292, 781129, 89969, 394365]
>> 	328, 	-2.551489695935748
>> 	8811, 	-2.4998450646849495
>> 	8518, 	-2.50162885308015
>> 	255584, 	-0.9974873389212762
>> 	9212, 	-2.4974037706696754
AVG ST.Z: 2.20957094465836
AVG SQ ST.Z: 376356.6758350165


In [223]:
# FAVORITES
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['favorites'])[0])
print(insyn_list)
out_list = ['Gun Samurai Recap', 'Marches Comes in Like a Lion: Special Omnibus Episode',
            'Berserk: Recollections of the Witch', 'So, I Can\'t Play H!',
            'Tsukigakirei: First Half - The Road So Far']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['favorites'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['favorites']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[70555, 166, 97, 88, 8670, 407, 3662]
>> 	0, 	-1.923879033114626
>> 	8, 	-1.9225910036346283
>> 	13, 	-1.9217859852096295
>> 	701, 	-1.8110154499298157
>> 	10, 	-1.9222689962646287
AVG ST.Z: 1.9003080936306656
AVG SQ ST.Z: 11806.143528878927


In [224]:
# FAVORITES
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Action'])[0])
print(insyn_list)
out_list = ['Gun Samurai Recap', 'Marches Comes in Like a Lion: Special Omnibus Episode',
            'Berserk: Recollections of the Witch', 'So, I Can\'t Play H!',
            'Tsukigakirei: First Half - The Road So Far']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Action'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['genre_Action']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[1, 1, 1, 1, 1, 0, 1]
>> 	0, 	-1.7546380429914956
>> 	0, 	-1.7546380429914956
>> 	1, 	0.29243967383191605
>> 	0, 	-1.7546380429914956
>> 	0, 	-1.7546380429914956
AVG ST.Z: 1.4621983691595797
AVG SQ ST.Z: 0.769309258162072
ABS MEAN DIFF: 0.657142857142857


In [225]:
# FAVORITES
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Comedy'])[0])
print(insyn_list)
out_list = ['Gun Samurai Recap', 'Marches Comes in Like a Lion: Special Omnibus Episode',
            'Berserk: Recollections of the Witch', 'So, I Can\'t Play H!',
            'Tsukigakirei: First Half - The Road So Far']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Comedy'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['genre_Comedy']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[0, 0, 0, 0, 0, 1, 0]
>> 	0, 	-0.28718058231174237
>> 	0, 	-0.28718058231174237
>> 	0, 	-0.28718058231174237
>> 	1, 	1.7230834938704545
>> 	0, 	-0.28718058231174237
AVG ST.Z: 0.5743611646234849
AVG SQ ST.Z: 0.4040610178208843
ABS MEAN DIFF: 0.05714285714285716


In [226]:
# FAVORITES
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Mystery'])[0])
print(insyn_list)
out_list = ['Gun Samurai Recap', 'Marches Comes in Like a Lion: Special Omnibus Episode',
            'Berserk: Recollections of the Witch', 'So, I Can\'t Play H!',
            'Tsukigakirei: First Half - The Road So Far']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Mystery'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['genre_Mystery']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[1, 0, 0, 0, 1, 0, 1]
>> 	0, 	-1.4971562248554138
>> 	0, 	-1.4971562248554138
>> 	0, 	-1.4971562248554138
>> 	0, 	-1.4971562248554138
>> 	0, 	-1.4971562248554138
AVG ST.Z: 1.4971562248554138
AVG SQ ST.Z: 0.42857142857142855
ABS MEAN DIFF: 0.42857142857142855


In [227]:
# FAVORITES
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['studio_Madhouse'])[0])
print(insyn_list)
out_list = ['Gun Samurai Recap', 'Marches Comes in Like a Lion: Special Omnibus Episode',
            'Berserk: Recollections of the Witch', 'So, I Can\'t Play H!',
            'Tsukigakirei: First Half - The Road So Far']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['studio_Madhouse'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['studio_Madhouse']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[0, 0, 0, 0, 0, 0, 0]
>> 	0, 	0.0
>> 	0, 	0.0
>> 	0, 	0.0
>> 	0, 	0.0
>> 	0, 	0.0
AVG ST.Z: 0.0
AVG SQ ST.Z: 0.0
ABS MEAN DIFF: 0


In [39]:
# from data_modeling EXAMPLE 2 (Euclidean, normalized)
# ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
#  'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
#  'Attack on Titan Season 3'] input,
# ['Gun Samurai Recap', 'Marches Comes in Like a Lion: Special Omnibus Episode',
#  'Berserk: Recollections of the Witch', 'So, I Can\'t Play H!',
#  'Tsukigakirei: First Half - The Road So Far'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = list(out_df[out_df['title_english'].isin(in_list)]['synopsis'])
for syn in insyn_list:
    print(syn[:300]+"...")
out_list = ['Marches Comes in Like a Lion: Special Omnibus Episode', 
            'Persona 4 the Animation: The Factor of Hope', 
            'Fullmetal Alchemist: Premium OVA Collection', 'Shiki Specials',
            'Robot Girls Z']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(out_df[out_df['title_english'] == t]['synopsis'])[0])
for syn in outsyn_list:
    print('>> \t%s'%(syn[:300]+"..."))

Centuries ago, mankind was slaughtered to near extinction by monstrous humanoid creatures called titans, forcing humans to hide in fear behind enormous concentric walls. What makes these giants truly terrifying is that their taste for human flesh is not born out of hunger but what appears to be out ...
Recap of the events in Shingeki no Kyojin that happened prior to episode 14....
Recap of episodes 1-13....
Recap of episodes 14-25....
For centuries, humanity has been hunted by giant, mysterious predators known as the Titans. Three mighty walls—Wall Maria, Rose, and Sheena—provided peace and protection for humanity for over a hundred years. That peace, however, was shattered when the Colossus Titan and Armored Titan appeared and d...
On his first day of junior high, Eren Yeager comes face-to-face with a titan—and has his lunch stolen! From that day on, he holds a grudge against titans for taking his favorite food from him, a cheeseburger, vowing to eliminate their kind once and for all.

In [228]:
# SCORED_BY
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['scored_by'])[0])
print(insyn_list)
out_list = ['Marches Comes in Like a Lion: Special Omnibus Episode', 
            'Persona 4 the Animation: The Factor of Hope', 
            'Fullmetal Alchemist: Premium OVA Collection', 'Shiki Specials',
            'Robot Girls Z']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['scored_by'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['scored_by']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[1038161, 36288, 20514, 15438, 439965, 43670, 154179]
>> 	4589, 	-2.536569782554626
>> 	3261, 	-2.550310277061114
>> 	27630, 	-2.298170133515479
>> 	28578, 	-2.2883614070153655
>> 	6135, 	-2.5205736948234287
AVG ST.Z: 2.4387970589940027
AVG SQ ST.Z: 235987.95469514964


In [229]:
# POPULARITY
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['popularity'])[0])
print(insyn_list)
out_list = ['Marches Comes in Like a Lion: Special Omnibus Episode', 
            'Persona 4 the Animation: The Factor of Hope', 
            'Fullmetal Alchemist: Premium OVA Collection', 'Shiki Specials',
            'Robot Girls Z']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['popularity'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['popularity']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[2, 1349, 1608, 1714, 41, 1050, 159]
>> 	4185, 	1.1641555349776394
>> 	3487, 	0.9207846662500703
>> 	1605, 	0.26458985973820043
>> 	1701, 	0.29806207090990045
>> 	3234, 	0.8325714430579857
AVG ST.Z: 0.6960327149867592
AVG SQ ST.Z: 2241.9009836315618


In [230]:
# EPISODES
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['episodes'])[0])
print(insyn_list)
out_list = ['Marches Comes in Like a Lion: Special Omnibus Episode', 
            'Persona 4 the Animation: The Factor of Hope', 
            'Fullmetal Alchemist: Premium OVA Collection', 'Shiki Specials',
            'Robot Girls Z']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['episodes'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['episodes']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[25.0, 1.0, 1.0, 1.0, 12.0, 12.0, 12.0]
>> 	1.0, 	-0.18119653709957603
>> 	1.0, 	-0.18119653709957603
>> 	3.0, 	-0.13669212447862752
>> 	2.0, 	-0.15894433078910178
>> 	3.0, 	-0.13669212447862752
AVG ST.Z: 0.15894433078910178
AVG SQ ST.Z: 7.1986393272107545


In [231]:
# RANK
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['rank'])[0])
print(insyn_list)
out_list = ['Marches Comes in Like a Lion: Special Omnibus Episode', 
            'Persona 4 the Animation: The Factor of Hope', 
            'Fullmetal Alchemist: Premium OVA Collection', 'Shiki Specials',
            'Robot Girls Z']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['rank'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['rank']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[116, 3529, 1166, 1002, 143, 3454, 113]
>> 	4747, 	1.2373887399275059
>> 	1879, 	0.1894761295004154
>> 	2000, 	0.23368722638384565
>> 	749, 	-0.22340436205558611
>> 	5800, 	1.6221348971031426
AVG ST.Z: 0.7012182709940992
AVG SQ ST.Z: 2538.9160422093037


In [232]:
# MEMBERS
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['members'])[0])
print(insyn_list)
out_list = ['Marches Comes in Like a Lion: Special Omnibus Episode', 
            'Persona 4 the Animation: The Factor of Hope', 
            'Fullmetal Alchemist: Premium OVA Collection', 'Shiki Specials',
            'Robot Girls Z']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['members'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['members']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[1500958, 68519, 52767, 48292, 781129, 89969, 394365]
>> 	8811, 	-2.4998450646849495
>> 	13700, 	-2.4700807593466076
>> 	52727, 	-2.232483797914932
>> 	48923, 	-2.255642606977532
>> 	16045, 	-2.455804364170005
AVG ST.Z: 2.382771318618805
AVG SQ ST.Z: 391837.8825894501


In [233]:
# FAVORITES
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['favorites'])[0])
print(insyn_list)
out_list = ['Marches Comes in Like a Lion: Special Omnibus Episode', 
            'Persona 4 the Animation: The Factor of Hope', 
            'Fullmetal Alchemist: Premium OVA Collection', 'Shiki Specials',
            'Robot Girls Z']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['favorites'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['favorites']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[70555, 166, 97, 88, 8670, 407, 3662]
>> 	8, 	-1.9225910036346283
>> 	15, 	-1.92146397783963
>> 	101, 	-1.9076176609296533
>> 	76, 	-1.9116427530546467
>> 	14, 	-1.9216249815246298
AVG ST.Z: 1.9169880753966375
AVG SQ ST.Z: 11906.54704876648


In [234]:
# FAVORITES
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Action'])[0])
print(insyn_list)
out_list = ['Marches Comes in Like a Lion: Special Omnibus Episode', 
            'Persona 4 the Animation: The Factor of Hope', 
            'Fullmetal Alchemist: Premium OVA Collection', 'Shiki Specials',
            'Robot Girls Z']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Action'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['genre_Action']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[1, 1, 1, 1, 1, 0, 1]
>> 	0, 	-1.7546380429914956
>> 	0, 	-1.7546380429914956
>> 	0, 	-1.7546380429914956
>> 	0, 	-1.7546380429914956
>> 	0, 	-1.7546380429914956
AVG ST.Z: 1.7546380429914954
AVG SQ ST.Z: 0.8571428571428571
ABS MEAN DIFF: 0.8571428571428571


In [235]:
# FAVORITES
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Comedy'])[0])
print(insyn_list)
out_list = ['Marches Comes in Like a Lion: Special Omnibus Episode', 
            'Persona 4 the Animation: The Factor of Hope', 
            'Fullmetal Alchemist: Premium OVA Collection', 'Shiki Specials',
            'Robot Girls Z']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Comedy'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['genre_Comedy']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[0, 0, 0, 0, 0, 1, 0]
>> 	0, 	-0.28718058231174237
>> 	0, 	-0.28718058231174237
>> 	1, 	1.7230834938704545
>> 	0, 	-0.28718058231174237
>> 	1, 	1.7230834938704545
AVG ST.Z: 0.8615417469352271
AVG SQ ST.Z: 0.5532833351724882
ABS MEAN DIFF: 0.2571428571428572


In [236]:
# FAVORITES
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Mystery'])[0])
print(insyn_list)
out_list = ['Marches Comes in Like a Lion: Special Omnibus Episode', 
            'Persona 4 the Animation: The Factor of Hope', 
            'Fullmetal Alchemist: Premium OVA Collection', 'Shiki Specials',
            'Robot Girls Z']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Mystery'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['genre_Mystery']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[1, 0, 0, 0, 1, 0, 1]
>> 	0, 	-1.4971562248554138
>> 	1, 	1.9962082998072184
>> 	0, 	-1.4971562248554138
>> 	1, 	1.9962082998072184
>> 	0, 	-1.4971562248554138
AVG ST.Z: 1.6967770548361358
AVG SQ ST.Z: 0.490730401066219
ABS MEAN DIFF: 0.028571428571428525


In [237]:
# FAVORITES
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['studio_Madhouse'])[0])
print(insyn_list)
out_list = ['Marches Comes in Like a Lion: Special Omnibus Episode', 
            'Persona 4 the Animation: The Factor of Hope', 
            'Fullmetal Alchemist: Premium OVA Collection', 'Shiki Specials',
            'Robot Girls Z']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['studio_Madhouse'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['studio_Madhouse']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[0, 0, 0, 0, 0, 0, 0]
>> 	0, 	0.0
>> 	0, 	0.0
>> 	0, 	0.0
>> 	0, 	0.0
>> 	0, 	0.0
AVG ST.Z: 0.0
AVG SQ ST.Z: 0.0
ABS MEAN DIFF: 0


In [46]:
# from data_modeling EXAMPLE 2 (Euclidean, un-normalized)
# ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
#  'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
#  'Attack on Titan Season 3'] input,
# ['Gun Samurai Recap', 'Marches Comes in Like a Lion: Special Omnibus Episode',
#  'Berserk: Recollections of the Witch', 'So, I Can\'t Play H!',
#  'Tsukigakirei: First Half - The Road So Far'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = list(out_df[out_df['title_english'].isin(in_list)]['synopsis'])
for syn in insyn_list:
    print(syn[:300]+"...")
out_list = ['Miss Kobayashi\'s Dragon Maid', 'Rosario + Vampire', 
            'My Teen Romantic Comedy SNAFU TOO!', 'GATE',
            'JoJo\'s Bizarre Adventure']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(out_df[out_df['title_english'] == t]['synopsis'])[0])
for syn in outsyn_list:
    print('>> \t%s'%(syn[:300]+"..."))

Centuries ago, mankind was slaughtered to near extinction by monstrous humanoid creatures called titans, forcing humans to hide in fear behind enormous concentric walls. What makes these giants truly terrifying is that their taste for human flesh is not born out of hunger but what appears to be out ...
Recap of the events in Shingeki no Kyojin that happened prior to episode 14....
Recap of episodes 1-13....
Recap of episodes 14-25....
For centuries, humanity has been hunted by giant, mysterious predators known as the Titans. Three mighty walls—Wall Maria, Rose, and Sheena—provided peace and protection for humanity for over a hundred years. That peace, however, was shattered when the Colossus Titan and Armored Titan appeared and d...
On his first day of junior high, Eren Yeager comes face-to-face with a titan—and has his lunch stolen! From that day on, he holds a grudge against titans for taking his favorite food from him, a cheeseburger, vowing to eliminate their kind once and for all.

In [238]:
# SCORED_BY
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['scored_by'])[0])
print(insyn_list)
out_list = ['Miss Kobayashi\'s Dragon Maid', 'Rosario + Vampire', 
            'My Teen Romantic Comedy SNAFU TOO!', 'GATE',
            'JoJo\'s Bizarre Adventure']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['scored_by'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['scored_by']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[1038161, 36288, 20514, 15438, 439965, 43670, 154179]
>> 	244021, 	-0.05922484228549446
>> 	245586, 	-0.043032166153978244
>> 	251353, 	0.016637586721711232
>> 	235214, 	-0.15034873921218028
>> 	14356, 	-2.435512998032889
AVG ST.Z: 0.5409512664812507
AVG SQ ST.Z: 105519.45640781135


In [239]:
# POPULARITY
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['popularity'])[0])
print(insyn_list)
out_list = ['Miss Kobayashi\'s Dragon Maid', 'Rosario + Vampire', 
            'My Teen Romantic Comedy SNAFU TOO!', 'GATE',
            'JoJo\'s Bizarre Adventure']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['popularity'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['popularity']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[2, 1349, 1608, 1714, 41, 1050, 159]
>> 	134, 	-0.2483020426948285
>> 	143, 	-0.24516402289748163
>> 	148, 	-0.24342067856562224
>> 	139, 	-0.24655869836296912
>> 	2139, 	0.45077903438078193
AVG ST.Z: 0.2868448953803367
AVG SQ ST.Z: 855.6274174843979


In [240]:
# EPISODES
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['episodes'])[0])
print(insyn_list)
out_list = ['Miss Kobayashi\'s Dragon Maid', 'Rosario + Vampire', 
            'My Teen Romantic Comedy SNAFU TOO!', 'GATE',
            'JoJo\'s Bizarre Adventure']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['episodes'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['episodes']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[25.0, 1.0, 1.0, 1.0, 12.0, 12.0, 12.0]
>> 	13.0, 	0.08582993862611497
>> 	13.0, 	0.08582993862611497
>> 	13.0, 	0.08582993862611497
>> 	12.0, 	0.06357773231564072
>> 	7.0, 	-0.04768329923673052
AVG ST.Z: 0.07375016948614324
AVG SQ ST.Z: 3.387853453207232


In [241]:
# RANK
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['rank'])[0])
print(insyn_list)
out_list = ['Miss Kobayashi\'s Dragon Maid', 'Rosario + Vampire', 
            'My Teen Romantic Comedy SNAFU TOO!', 'GATE',
            'JoJo\'s Bizarre Adventure']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['rank'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['rank']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[116, 3529, 1166, 1002, 143, 3454, 113]
>> 	359, 	-0.36590293878730346
>> 	3569, 	0.8069699620045238
>> 	271, 	-0.39805646379343457
>> 	758, 	-0.22011593336177726
>> 	2818, 	0.5325688565544733
AVG ST.Z: 0.4647228309003024
AVG SQ ST.Z: 1382.3846211587881


In [242]:
# MEMBERS
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['members'])[0])
print(insyn_list)
out_list = ['Miss Kobayashi\'s Dragon Maid', 'Rosario + Vampire', 
            'My Teen Romantic Comedy SNAFU TOO!', 'GATE',
            'JoJo\'s Bizarre Adventure']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['members'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['members']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[1500958, 68519, 52767, 48292, 781129, 89969, 394365]
>> 	426274, 	0.041675941545393294
>> 	414882, 	-0.027678725342393323
>> 	405743, 	-0.08331709443363998
>> 	419943, 	0.0031327185760659532
>> 	34055, 	-2.346159214007695
AVG ST.Z: 0.5003927387810375
AVG SQ ST.Z: 172492.18361930232


In [243]:
# FAVORITES
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['favorites'])[0])
print(insyn_list)
out_list = ['Miss Kobayashi\'s Dragon Maid', 'Rosario + Vampire', 
            'My Teen Romantic Comedy SNAFU TOO!', 'GATE',
            'JoJo\'s Bizarre Adventure']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['favorites'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['favorites']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[70555, 166, 97, 88, 8670, 407, 3662]
>> 	7684, 	-0.6867267175767038
>> 	3262, 	-1.3986850126455082
>> 	10758, 	-0.19180138988753495
>> 	4266, 	-1.2370373129057795
>> 	166, 	-1.8971524214046709
AVG ST.Z: 1.0822805708840395
AVG SQ ST.Z: 7654.534062253827


In [244]:
# for 'genre_Action'
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Action'])[0])
print(insyn_list)
out_list = ['Miss Kobayashi\'s Dragon Maid', 'Rosario + Vampire', 
            'My Teen Romantic Comedy SNAFU TOO!', 'GATE',
            'JoJo\'s Bizarre Adventure']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Action'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['genre_Action']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[1, 1, 1, 1, 1, 0, 1]
>> 	0, 	-1.7546380429914956
>> 	0, 	-1.7546380429914956
>> 	0, 	-1.7546380429914956
>> 	1, 	0.29243967383191605
>> 	0, 	-1.7546380429914956
AVG ST.Z: 1.4621983691595797
AVG SQ ST.Z: 0.769309258162072
ABS MEAN DIFF: 0.657142857142857


In [245]:
# for 'genre_Action'
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Comedy'])[0])
print(insyn_list)
out_list = ['Miss Kobayashi\'s Dragon Maid', 'Rosario + Vampire', 
            'My Teen Romantic Comedy SNAFU TOO!', 'GATE',
            'JoJo\'s Bizarre Adventure']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Comedy'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['genre_Comedy']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[0, 0, 0, 0, 0, 1, 0]
>> 	1, 	1.7230834938704545
>> 	1, 	1.7230834938704545
>> 	1, 	1.7230834938704545
>> 	0, 	-0.28718058231174237
>> 	0, 	-0.28718058231174237
AVG ST.Z: 1.1487223292469695
AVG SQ ST.Z: 0.67005939426049
ABS MEAN DIFF: 0.45714285714285713


In [246]:
# for 'genre_Action'
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Mystery'])[0])
print(insyn_list)
out_list = ['Miss Kobayashi\'s Dragon Maid', 'Rosario + Vampire', 
            'My Teen Romantic Comedy SNAFU TOO!', 'GATE',
            'JoJo\'s Bizarre Adventure']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Mystery'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['genre_Mystery']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[1, 0, 0, 0, 1, 0, 1]
>> 	0, 	-1.4971562248554138
>> 	0, 	-1.4971562248554138
>> 	0, 	-1.4971562248554138
>> 	0, 	-1.4971562248554138
>> 	0, 	-1.4971562248554138
AVG ST.Z: 1.4971562248554138
AVG SQ ST.Z: 0.42857142857142855
ABS MEAN DIFF: 0.42857142857142855


In [247]:
# for 'genre_Action'
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['studio_Madhouse'])[0])
print(insyn_list)
out_list = ['Miss Kobayashi\'s Dragon Maid', 'Rosario + Vampire', 
            'My Teen Romantic Comedy SNAFU TOO!', 'GATE',
            'JoJo\'s Bizarre Adventure']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['studio_Madhouse'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['studio_Madhouse']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[0, 0, 0, 0, 0, 0, 0]
>> 	0, 	0.0
>> 	0, 	0.0
>> 	0, 	0.0
>> 	0, 	0.0
>> 	0, 	0.0
AVG ST.Z: 0.0
AVG SQ ST.Z: 0.0
ABS MEAN DIFF: 0


In [75]:
# Cosine un-normalized KNN
# results for ['Queen\'s Blade: Inheritor of the Throne']
med = median(list(gen_df['popularity']))
print(med)
rec_indices = [2598, 1922, 1757, 2240, 376]
dat = gen_df[gen_df.index.isin(rec_indices)]
pop_dat = list(dat['popularity'])

sumz = 0
for p in pop_dat:
    z = (std_z(p, med, f_stdv['popularity']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(p,z))
    
print("AVG ST.Z: %s"%(sumz/len(rec_indices)))
print("AVG SQ ST.Z: %s"%(avg_std_z(pop_dat,med)))

1975
>> 	2018, 	0.014992761253990648
>> 	1788, 	-0.06520107801154072
>> 	1795, 	-0.0627603959469376
>> 	2281, 	0.10669267310979391
>> 	1980, 	0.0017433443318593776
AVG ST.Z: 0.050278050530824445
AVG SQ ST.Z: 180.48767270924625


In [76]:
# from data_modeling EXAMPLE 3 (Cosine, un-normalized)
in_list = ['Attack on Titan', 'Attack on Titan Season 2', 
           'Bungo Stray Dogs', 'My Hero Academia 3', 'Nanbaka',
           'Nanbaka: Season 2', 'Nanbaka: Idiots with Student Numbers!',
           'One Punch Man']
insyn_list = list(out_df[out_df['title_english'].isin(in_list)]['synopsis'])
for syn in insyn_list:
    print(syn[:300]+"...")
out_list = ['Fullmetal Alchemist', 'The Future Diary', 'Elfen Lied', 
            'Parasyte -the maxim-', 'My Teen Romantic Comedy SNAFU TOO!']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(out_df[out_df['title_english'] == t]['synopsis'])[0])
for syn in outsyn_list:
    print('>> \t%s'%(syn[:300]+"..."))

Centuries ago, mankind was slaughtered to near extinction by monstrous humanoid creatures called titans, forcing humans to hide in fear behind enormous concentric walls. What makes these giants truly terrifying is that their taste for human flesh is not born out of hunger but what appears to be out ...
For centuries, humanity has been hunted by giant, mysterious predators known as the Titans. Three mighty walls—Wall Maria, Rose, and Sheena—provided peace and protection for humanity for over a hundred years. That peace, however, was shattered when the Colossus Titan and Armored Titan appeared and d...
Nanba is the world's most formidable prison, built to incarcerate criminals who are too slippery to stay in ordinary confinement. The four inmates who occupy Cell 13 are particularly cunning on that behalf, having escaped every other prison with a perfect success rate. There is Juugo, a specialist i...
The seemingly ordinary and unimpressive Saitama has a rather unique hobby: being a hero.

In [248]:
# EXAMPLE 3, scored_by

in_list = ['Attack on Titan', 'Attack on Titan Season 2', 
   'Bungo Stray Dogs', 'My Hero Academia 3', 'Nanbaka', 'Nanbaka: Season 2', 'Nanbaka: Idiots with Student Numbers!',
   'One Punch Man']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['scored_by'])[0])
print(insyn_list)
out_list = ['Fullmetal Alchemist', 'The Future Diary', 'Elfen Lied', 'Parasyte -the maxim-', 
            'My Teen Romantic Comedy SNAFU TOO!']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['scored_by'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['scored_by']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[1038161, 439965, 190064, 262182, 57332, 26862, 5131, 806647]
>> 	524986, 	1.776465906101223
>> 	648490, 	3.054331895204596
>> 	549257, 	2.0275920673715295
>> 	463232, 	1.1375122180337134
>> 	251353, 	-1.0547485014995293
AVG ST.Z: 1.810130117642118
AVG SQ ST.Z: 188414.43239571643


In [249]:
# EXAMPLE 3, popularity

in_list = ['Attack on Titan', 'Attack on Titan Season 2', 
   'Bungo Stray Dogs', 'My Hero Academia 3', 'Nanbaka', 'Nanbaka: Season 2', 'Nanbaka: Idiots with Student Numbers!',
   'One Punch Man']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['popularity'])[0])
print(insyn_list)
out_list = ['Fullmetal Alchemist', 'The Future Diary', 'Elfen Lied', 'Parasyte -the maxim-', 
            'My Teen Romantic Comedy SNAFU TOO!']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['popularity'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['popularity']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[2, 41, 152, 89, 658, 1534, 3340, 5]
>> 	27, 	-0.2442861245017953
>> 	12, 	-0.24951615749737344
>> 	18, 	-0.24742414429914217
>> 	44, 	-0.2383587537734734
>> 	148, 	-0.20209719167079834
AVG ST.Z: 0.2363364743485165
AVG SQ ST.Z: 679.6867591949986


In [250]:
# EXAMPLE 3, episodes

in_list = ['Attack on Titan', 'Attack on Titan Season 2', 
   'Bungo Stray Dogs', 'My Hero Academia 3', 'Nanbaka', 'Nanbaka: Season 2', 'Nanbaka: Idiots with Student Numbers!',
   'One Punch Man']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['episodes'])[0])
print(insyn_list)
out_list = ['Fullmetal Alchemist', 'The Future Diary', 'Elfen Lied', 'Parasyte -the maxim-', 
            'My Teen Romantic Comedy SNAFU TOO!']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['episodes'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['episodes']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[25.0, 12.0, 12.0, 25.0, 13.0, 12.0, 1.0, 12.0]
>> 	51.0, 	0.8233316334875472
>> 	26.0, 	0.267026475725691
>> 	13.0, 	-0.02225220631047425
>> 	24.0, 	0.2225220631047425
>> 	13.0, 	-0.02225220631047425
AVG ST.Z: 0.2714769169877858
AVG SQ ST.Z: 17.97220075561143


In [251]:
# EXAMPLE 3, rank

in_list = ['Attack on Titan', 'Attack on Titan Season 2', 
   'Bungo Stray Dogs', 'My Hero Academia 3', 'Nanbaka', 'Nanbaka: Season 2', 'Nanbaka: Idiots with Student Numbers!',
   'One Punch Man']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['rank'])[0])
print(insyn_list)
out_list = ['Fullmetal Alchemist', 'The Future Diary', 'Elfen Lied', 'Parasyte -the maxim-', 
            'My Teen Romantic Comedy SNAFU TOO!']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['rank'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['rank']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[116, 143, 858, 50, 1972, 1347, 4026, 46]
>> 	268, 	-0.2929441894734728
>> 	824, 	-0.08979237238928087
>> 	1067, 	-0.0010047976564415966
>> 	104, 	-0.3528666678939898
>> 	271, 	-0.2918480465755365
AVG ST.Z: 0.20569121479774427
AVG SQ ST.Z: 674.3705676406704


In [252]:
# EXAMPLE 3, members

in_list = ['Attack on Titan', 'Attack on Titan Season 2', 
   'Bungo Stray Dogs', 'My Hero Academia 3', 'Nanbaka', 'Nanbaka: Season 2', 'Nanbaka: Idiots with Student Numbers!',
   'One Punch Man']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['members'])[0])
print(insyn_list)
out_list = ['Fullmetal Alchemist', 'The Future Diary', 'Elfen Lied', 'Parasyte -the maxim-', 
            'My Teen Romantic Comedy SNAFU TOO!']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['members'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['members']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[1500958, 781129, 401847, 535390, 143814, 57054, 15109, 1195384]
>> 	854022, 	1.6753387789837193
>> 	1064223, 	2.9550456201423954
>> 	901294, 	1.9631314240960305
>> 	767501, 	1.1485976331095809
>> 	405743, 	-1.0537904975076866
AVG ST.Z: 1.7591807907678825
AVG SQ ST.Z: 310117.2452334611


In [253]:
# EXAMPLE 3, favorites

in_list = ['Attack on Titan', 'Attack on Titan Season 2', 
   'Bungo Stray Dogs', 'My Hero Academia 3', 'Nanbaka', 'Nanbaka: Season 2', 'Nanbaka: Idiots with Student Numbers!',
   'One Punch Man']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['favorites'])[0])
print(insyn_list)
out_list = ['Fullmetal Alchemist', 'The Future Diary', 'Elfen Lied', 'Parasyte -the maxim-', 
            'My Teen Romantic Comedy SNAFU TOO!']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['favorites'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['favorites']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[70555, 8670, 5777, 8361, 971, 232, 30, 35969]
>> 	22764, 	1.0374071188351328
>> 	25002, 	1.3977333658645277
>> 	20990, 	0.7517865816456124
>> 	18100, 	0.2864859319963939
>> 	10758, 	-0.895603123271621
AVG ST.Z: 0.8738032243226576
AVG SQ ST.Z: 5878.69693815092


In [254]:
# EXAMPLE 3, genre_Comedy

in_list = ['Attack on Titan', 'Attack on Titan Season 2', 
   'Bungo Stray Dogs', 'My Hero Academia 3', 'Nanbaka', 'Nanbaka: Season 2', 'Nanbaka: Idiots with Student Numbers!',
   'One Punch Man']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Comedy'])[0])
print(insyn_list)
out_list = ['Fullmetal Alchemist', 'The Future Diary', 'Elfen Lied', 'Parasyte -the maxim-', 
            'My Teen Romantic Comedy SNAFU TOO!']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Comedy'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['genre_Comedy']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[0, 0, 1, 1, 1, 1, 1, 1]
>> 	1, 	0.5025660190455492
>> 	0, 	-1.5076980571366474
>> 	0, 	-1.5076980571366474
>> 	0, 	-1.5076980571366474
>> 	1, 	0.5025660190455492
AVG ST.Z: 1.105645241900208
AVG SQ ST.Z: 0.6020797289396148
ABS MEAN DIFF: 0.35


In [255]:
# EXAMPLE 3, genre_Action

in_list = ['Attack on Titan', 'Attack on Titan Season 2', 
   'Bungo Stray Dogs', 'My Hero Academia 3', 'Nanbaka', 'Nanbaka: Season 2', 'Nanbaka: Idiots with Student Numbers!',
   'One Punch Man']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Action'])[0])
print(insyn_list)
out_list = ['Fullmetal Alchemist', 'The Future Diary', 'Elfen Lied', 'Parasyte -the maxim-', 
            'My Teen Romantic Comedy SNAFU TOO!']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Action'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['genre_Action']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[1, 1, 1, 1, 1, 1, 1, 1]
>> 	1, 	0.0
>> 	1, 	0.0
>> 	1, 	0.0
>> 	1, 	0.0
>> 	0, 	-2.0470777168234116
AVG ST.Z: 0.40941554336468233
AVG SQ ST.Z: 0.4472135954999579
ABS MEAN DIFF: 0.19999999999999996


In [85]:
# from data_modeling EXAMPLE 3 (Cosine, normalized)
in_list = ['Attack on Titan', 'Attack on Titan Season 2', 
           'Bungo Stray Dogs', 'My Hero Academia 3', 'Nanbaka',
           'Nanbaka: Season 2', 'Nanbaka: Idiots with Student Numbers!',
           'One Punch Man']
insyn_list = list(out_df[out_df['title_english'].isin(in_list)]['synopsis'])
for syn in insyn_list:
    print(syn[:300]+"...")
out_list = ['Fullmetal Alchemist: Brotherhood', 'My Hero Academia', 
            'Code Geass: Lelouch of the Rebellion', 'Death Note', 
            'Code Geass: Lelouch of the Rebellion R2']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(out_df[out_df['title_english'] == t]['synopsis'])[0])
for syn in outsyn_list:
    print('>> \t%s'%(syn[:300]+"..."))

Centuries ago, mankind was slaughtered to near extinction by monstrous humanoid creatures called titans, forcing humans to hide in fear behind enormous concentric walls. What makes these giants truly terrifying is that their taste for human flesh is not born out of hunger but what appears to be out ...
For centuries, humanity has been hunted by giant, mysterious predators known as the Titans. Three mighty walls—Wall Maria, Rose, and Sheena—provided peace and protection for humanity for over a hundred years. That peace, however, was shattered when the Colossus Titan and Armored Titan appeared and d...
Nanba is the world's most formidable prison, built to incarcerate criminals who are too slippery to stay in ordinary confinement. The four inmates who occupy Cell 13 are particularly cunning on that behalf, having escaped every other prison with a perfect success rate. There is Juugo, a specialist i...
The seemingly ordinary and unimpressive Saitama has a rather unique hobby: being a hero.

In [256]:
# EXAMPLE 3, scored_by

in_list = ['Attack on Titan', 'Attack on Titan Season 2', 
   'Bungo Stray Dogs', 'My Hero Academia 3', 'Nanbaka', 'Nanbaka: Season 2', 'Nanbaka: Idiots with Student Numbers!',
   'One Punch Man']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['scored_by'])[0])
print(insyn_list)
out_list = ['Fullmetal Alchemist: Brotherhood', 'My Hero Academia', 
            'Code Geass: Lelouch of the Rebellion', 'Death Note', 
            'Code Geass: Lelouch of the Rebellion R2']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['scored_by'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['scored_by']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[1038161, 439965, 190064, 262182, 57332, 26862, 5131, 806647]
>> 	826899, 	4.900286627439534
>> 	625249, 	2.8138628945831465
>> 	681621, 	3.397130331570899
>> 	1107955, 	7.808305019017439
>> 	591272, 	2.4623111010237047
AVG ST.Z: 4.276379194726944
AVG SQ ST.Z: 454359.0482076923


In [257]:
# EXAMPLE 3, popularity

in_list = ['Attack on Titan', 'Attack on Titan Season 2', 
   'Bungo Stray Dogs', 'My Hero Academia 3', 'Nanbaka', 'Nanbaka: Season 2', 'Nanbaka: Idiots with Student Numbers!',
   'One Punch Man']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['popularity'])[0])
print(insyn_list)
out_list = ['Fullmetal Alchemist: Brotherhood', 'My Hero Academia', 
            'Code Geass: Lelouch of the Rebellion', 'Death Note', 
            'Code Geass: Lelouch of the Rebellion R2']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['popularity'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['popularity']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[2, 41, 152, 89, 658, 1534, 3340, 5]
>> 	4, 	-0.2523055084283484
>> 	15, 	-0.2484701508982578
>> 	11, 	-0.2498648263637453
>> 	1, 	-0.25335151502746406
>> 	24, 	-0.24533213110091093
AVG ST.Z: 0.2498648263637453
AVG SQ ST.Z: 716.6716058453829


In [258]:
# EXAMPLE 3, episodes

in_list = ['Attack on Titan', 'Attack on Titan Season 2', 
   'Bungo Stray Dogs', 'My Hero Academia 3', 'Nanbaka', 'Nanbaka: Season 2', 'Nanbaka: Idiots with Student Numbers!',
   'One Punch Man']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['episodes'])[0])
print(insyn_list)
out_list = ['Fullmetal Alchemist: Brotherhood', 'My Hero Academia', 
            'Code Geass: Lelouch of the Rebellion', 'Death Note', 
            'Code Geass: Lelouch of the Rebellion R2']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['episodes'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['episodes']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[25.0, 12.0, 12.0, 25.0, 13.0, 12.0, 1.0, 12.0]
>> 	64.0, 	1.1126103155237126
>> 	13.0, 	-0.02225220631047425
>> 	25.0, 	0.24477426941521674
>> 	37.0, 	0.5118007451409078
>> 	25.0, 	0.24477426941521674
AVG ST.Z: 0.4272423611611056
AVG SQ ST.Z: 25.581243128511172


In [259]:
# EXAMPLE 3, rank

in_list = ['Attack on Titan', 'Attack on Titan Season 2', 
   'Bungo Stray Dogs', 'My Hero Academia 3', 'Nanbaka', 'Nanbaka: Season 2', 'Nanbaka: Idiots with Student Numbers!',
   'One Punch Man']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['rank'])[0])
print(insyn_list)
out_list = ['Fullmetal Alchemist: Brotherhood', 'My Hero Academia', 
            'Code Geass: Lelouch of the Rebellion', 'Death Note', 
            'Code Geass: Lelouch of the Rebellion R2']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['rank'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['rank']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[116, 143, 858, 50, 1972, 1347, 4026, 46]
>> 	1, 	-0.3905009073898023
>> 	152, 	-0.3353283815270092
>> 	32, 	-0.3791740974444607
>> 	54, 	-0.3711357161929279
>> 	17, 	-0.38465481193414214
AVG ST.Z: 0.37215878289766846
AVG SQ ST.Z: 1019.9458135116787


In [260]:
# EXAMPLE 3, members

in_list = ['Attack on Titan', 'Attack on Titan Season 2', 
   'Bungo Stray Dogs', 'My Hero Academia 3', 'Nanbaka', 'Nanbaka: Season 2', 'Nanbaka: Idiots with Student Numbers!',
   'One Punch Man']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['members'])[0])
print(insyn_list)
out_list = ['Fullmetal Alchemist: Brotherhood', 'My Hero Academia', 
            'Code Geass: Lelouch of the Rebellion', 'Death Note', 
            'Code Geass: Lelouch of the Rebellion R2']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['members'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['members']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[1500958, 781129, 401847, 535390, 143814, 57054, 15109, 1195384]
>> 	1355349, 	4.727425075231385
>> 	958412, 	2.3108666649050718
>> 	1082349, 	3.0653969800447847
>> 	1610561, 	6.281159559585826
>> 	869302, 	1.7683636481941636
AVG ST.Z: 3.630642385592246
AVG SQ ST.Z: 655653.0987100501


In [261]:
# EXAMPLE 3, favorites

in_list = ['Attack on Titan', 'Attack on Titan Season 2', 
   'Bungo Stray Dogs', 'My Hero Academia 3', 'Nanbaka', 'Nanbaka: Season 2', 'Nanbaka: Idiots with Student Numbers!',
   'One Punch Man']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['favorites'])[0])
print(insyn_list)
out_list = ['Fullmetal Alchemist: Brotherhood', 'My Hero Academia', 
            'Code Geass: Lelouch of the Rebellion', 'Death Note', 
            'Code Geass: Lelouch of the Rebellion R2']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['favorites'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['favorites']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[70555, 8670, 5777, 8361, 971, 232, 30, 35969]
>> 	120331, 	16.746053653203752
>> 	28657, 	1.9862018345385393
>> 	68684, 	8.430696334022716
>> 	96146, 	12.852179531485291
>> 	47637, 	5.042051775833407
AVG ST.Z: 9.01143662581674
AVG SQ ST.Z: 64907.73934470854


In [262]:
# EXAMPLE 3, genre_Comedy

in_list = ['Attack on Titan', 'Attack on Titan Season 2', 
   'Bungo Stray Dogs', 'My Hero Academia 3', 'Nanbaka', 'Nanbaka: Season 2', 'Nanbaka: Idiots with Student Numbers!',
   'One Punch Man']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Comedy'])[0])
print(insyn_list)
out_list = ['Fullmetal Alchemist: Brotherhood', 'My Hero Academia', 
            'Code Geass: Lelouch of the Rebellion', 'Death Note', 
            'Code Geass: Lelouch of the Rebellion R2']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Comedy'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['genre_Comedy']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[0, 0, 1, 1, 1, 1, 1, 1]
>> 	1, 	0.5025660190455492
>> 	1, 	0.5025660190455492
>> 	0, 	-1.5076980571366474
>> 	0, 	-1.5076980571366474
>> 	0, 	-1.5076980571366474
AVG ST.Z: 1.105645241900208
AVG SQ ST.Z: 0.6020797289396148
ABS MEAN DIFF: 0.35


In [263]:
# EXAMPLE 3, genre_Comedy

in_list = ['Attack on Titan', 'Attack on Titan Season 2', 
   'Bungo Stray Dogs', 'My Hero Academia 3', 'Nanbaka', 'Nanbaka: Season 2', 'Nanbaka: Idiots with Student Numbers!',
   'One Punch Man']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Action'])[0])
print(insyn_list)
out_list = ['Fullmetal Alchemist: Brotherhood', 'My Hero Academia', 
            'Code Geass: Lelouch of the Rebellion', 'Death Note', 
            'Code Geass: Lelouch of the Rebellion R2']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Action'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['genre_Action']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[1, 1, 1, 1, 1, 1, 1, 1]
>> 	1, 	0.0
>> 	1, 	0.0
>> 	1, 	0.0
>> 	0, 	-2.0470777168234116
>> 	1, 	0.0
AVG ST.Z: 0.40941554336468233
AVG SQ ST.Z: 0.4472135954999579
ABS MEAN DIFF: 0.19999999999999996


In [93]:
# from data_modeling EXAMPLE 3 (Euclidean, normalized)
in_list = ['Attack on Titan', 'Attack on Titan Season 2', 
           'Bungo Stray Dogs', 'My Hero Academia 3', 'Nanbaka',
           'Nanbaka: Season 2', 'Nanbaka: Idiots with Student Numbers!',
           'One Punch Man']
insyn_list = list(out_df[out_df['title_english'].isin(in_list)]['synopsis'])
for syn in insyn_list:
    print(syn[:300]+"...")
out_list = ['JoJo\'s Bizarre Adventure: Diamond is Unbreakable', 
            'Re:CREATORS', 'Akame ga Kill!', 'Drifters', 
            'JoJo\'s Bizarre Adventure: Stardust Crusaders']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(out_df[out_df['title_english'] == t]['synopsis'])[0])
for syn in outsyn_list:
    print('>> \t%s'%(syn[:300]+"..."))

Centuries ago, mankind was slaughtered to near extinction by monstrous humanoid creatures called titans, forcing humans to hide in fear behind enormous concentric walls. What makes these giants truly terrifying is that their taste for human flesh is not born out of hunger but what appears to be out ...
For centuries, humanity has been hunted by giant, mysterious predators known as the Titans. Three mighty walls—Wall Maria, Rose, and Sheena—provided peace and protection for humanity for over a hundred years. That peace, however, was shattered when the Colossus Titan and Armored Titan appeared and d...
Nanba is the world's most formidable prison, built to incarcerate criminals who are too slippery to stay in ordinary confinement. The four inmates who occupy Cell 13 are particularly cunning on that behalf, having escaped every other prison with a perfect success rate. There is Juugo, a specialist i...
The seemingly ordinary and unimpressive Saitama has a rather unique hobby: being a hero.

In [264]:
# EXAMPLE 3, scored_by

in_list = ['Attack on Titan', 'Attack on Titan Season 2', 
   'Bungo Stray Dogs', 'My Hero Academia 3', 'Nanbaka', 'Nanbaka: Season 2', 'Nanbaka: Idiots with Student Numbers!',
   'One Punch Man']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['scored_by'])[0])
print(insyn_list)
out_list = ['JoJo\'s Bizarre Adventure: Diamond is Unbreakable', 
            'Re:CREATORS', 'Akame ga Kill!', 'Drifters', 
            'JoJo\'s Bizarre Adventure: Stardust Crusaders']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['scored_by'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['scored_by']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[1038161, 439965, 190064, 262182, 57332, 26862, 5131, 806647]
>> 	134149, 	-2.267429915760377
>> 	90342, 	-2.7206903395899813
>> 	531506, 	1.843926767684281
>> 	133963, 	-2.2693544127319183
>> 	176466, 	-1.8295861612189253
AVG ST.Z: 2.186197519397097
AVG SQ ST.Z: 213682.30218480894


In [265]:
# EXAMPLE 3, popularity

in_list = ['Attack on Titan', 'Attack on Titan Season 2', 
   'Bungo Stray Dogs', 'My Hero Academia 3', 'Nanbaka', 'Nanbaka: Season 2', 'Nanbaka: Idiots with Student Numbers!',
   'One Punch Man']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['popularity'])[0])
print(insyn_list)
out_list = ['JoJo\'s Bizarre Adventure: Diamond is Unbreakable', 
            'Re:CREATORS', 'Akame ga Kill!', 'Drifters', 
            'JoJo\'s Bizarre Adventure: Stardust Crusaders']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['popularity'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['popularity']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[2, 41, 152, 89, 658, 1534, 3340, 5]
>> 	352, 	-0.13096874293093574
>> 	333, 	-0.13759345139200138
>> 	23, 	-0.2456807999672828
>> 	294, 	-0.15119153718050451
>> 	289, 	-0.1529348815123639
AVG ST.Z: 0.16367388259661766
AVG SQ ST.Z: 484.5084009849571


In [266]:
# EXAMPLE 3, episodes

in_list = ['Attack on Titan', 'Attack on Titan Season 2', 
   'Bungo Stray Dogs', 'My Hero Academia 3', 'Nanbaka', 'Nanbaka: Season 2', 'Nanbaka: Idiots with Student Numbers!',
   'One Punch Man']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['episodes'])[0])
print(insyn_list)
out_list = ['JoJo\'s Bizarre Adventure: Diamond is Unbreakable', 
            'Re:CREATORS', 'Akame ga Kill!', 'Drifters', 
            'JoJo\'s Bizarre Adventure: Stardust Crusaders']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['episodes'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['episodes']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[25.0, 12.0, 12.0, 25.0, 13.0, 12.0, 1.0, 12.0]
>> 	39.0, 	0.5563051577618563
>> 	22.0, 	0.178017650483794
>> 	24.0, 	0.2225220631047425
>> 	12.0, 	-0.0445044126209485
>> 	24.0, 	0.2225220631047425
AVG ST.Z: 0.24477426941521677
AVG SQ ST.Z: 13.36413109783049


In [267]:
# EXAMPLE 3, rank

in_list = ['Attack on Titan', 'Attack on Titan Season 2', 
   'Bungo Stray Dogs', 'My Hero Academia 3', 'Nanbaka', 'Nanbaka: Season 2', 'Nanbaka: Idiots with Student Numbers!',
   'One Punch Man']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['rank'])[0])
print(insyn_list)
out_list = ['JoJo\'s Bizarre Adventure: Diamond is Unbreakable', 
            'Re:CREATORS', 'Akame ga Kill!', 'Drifters', 
            'JoJo\'s Bizarre Adventure: Stardust Crusaders']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['rank'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['rank']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[116, 143, 858, 50, 1972, 1347, 4026, 46]
>> 	64, 	-0.3674819065331403
>> 	1216, 	0.053436966274394
>> 	946, 	-0.04521589453987185
>> 	524, 	-0.1994066621829096
>> 	291, 	-0.28454042725596124
AVG ST.Z: 0.1900163713572554
AVG SQ ST.Z: 624.904922768256


In [268]:
# EXAMPLE 3, members

in_list = ['Attack on Titan', 'Attack on Titan Season 2', 
   'Bungo Stray Dogs', 'My Hero Academia 3', 'Nanbaka', 'Nanbaka: Season 2', 'Nanbaka: Idiots with Student Numbers!',
   'One Punch Man']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['members'])[0])
print(insyn_list)
out_list = ['JoJo\'s Bizarre Adventure: Diamond is Unbreakable', 
            'Re:CREATORS', 'Akame ga Kill!', 'Drifters', 
            'JoJo\'s Bizarre Adventure: Stardust Crusaders']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['members'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['members']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[1500958, 781129, 401847, 535390, 143814, 57054, 15109, 1195384]
>> 	236502, 	-2.084132244238366
>> 	248516, 	-2.0109908320201537
>> 	874667, 	1.8010258486728306
>> 	275539, 	-1.8464744026566833
>> 	281554, 	-1.8098549924275718
AVG ST.Z: 1.9104956640031212
AVG SQ ST.Z: 314383.4522360244


In [269]:
# EXAMPLE 3, favorites

in_list = ['Attack on Titan', 'Attack on Titan Season 2', 
   'Bungo Stray Dogs', 'My Hero Academia 3', 'Nanbaka', 'Nanbaka: Season 2', 'Nanbaka: Idiots with Student Numbers!',
   'One Punch Man']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['favorites'])[0])
print(insyn_list)
out_list = ['JoJo\'s Bizarre Adventure: Diamond is Unbreakable', 
            'Re:CREATORS', 'Akame ga Kill!', 'Drifters', 
            'JoJo\'s Bizarre Adventure: Stardust Crusaders']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['favorites'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['favorites']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[70555, 8670, 5777, 8361, 971, 232, 30, 35969]
>> 	9765, 	-1.0554797824763524
>> 	2044, 	-2.2985892343592647
>> 	16107, 	-0.034394412208067236
>> 	2120, 	-2.2863529542992853
>> 	6335, 	-1.607722422025425
AVG ST.Z: 1.456507761073679
AVG SQ ST.Z: 10471.048975180329


In [270]:
# EXAMPLE 3, genre_Comedy

in_list = ['Attack on Titan', 'Attack on Titan Season 2', 
   'Bungo Stray Dogs', 'My Hero Academia 3', 'Nanbaka', 'Nanbaka: Season 2', 'Nanbaka: Idiots with Student Numbers!',
   'One Punch Man']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Comedy'])[0])
print(insyn_list)
out_list = ['JoJo\'s Bizarre Adventure: Diamond is Unbreakable', 
            'Re:CREATORS', 'Akame ga Kill!', 'Drifters', 
            'JoJo\'s Bizarre Adventure: Stardust Crusaders']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Comedy'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['genre_Comedy']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[0, 0, 1, 1, 1, 1, 1, 1]
>> 	1, 	0.5025660190455492
>> 	0, 	-1.5076980571366474
>> 	0, 	-1.5076980571366474
>> 	1, 	0.5025660190455492
>> 	0, 	-1.5076980571366474
AVG ST.Z: 1.105645241900208
AVG SQ ST.Z: 0.6020797289396148
ABS MEAN DIFF: 0.35


In [271]:
# EXAMPLE 3, genre_Action

in_list = ['Attack on Titan', 'Attack on Titan Season 2', 
   'Bungo Stray Dogs', 'My Hero Academia 3', 'Nanbaka', 'Nanbaka: Season 2', 'Nanbaka: Idiots with Student Numbers!',
   'One Punch Man']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Action'])[0])
print(insyn_list)
out_list = ['JoJo\'s Bizarre Adventure: Diamond is Unbreakable', 
            'Re:CREATORS', 'Akame ga Kill!', 'Drifters', 
            'JoJo\'s Bizarre Adventure: Stardust Crusaders']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Action'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['genre_Action']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[1, 1, 1, 1, 1, 1, 1, 1]
>> 	1, 	0.0
>> 	1, 	0.0
>> 	1, 	0.0
>> 	1, 	0.0
>> 	1, 	0.0
AVG ST.Z: 0.0
AVG SQ ST.Z: 0.0
ABS MEAN DIFF: 0


In [95]:
# from data_modeling EXAMPLE 3 (Euclidean, un-normalized)
in_list = ['Attack on Titan', 'Attack on Titan Season 2', 
           'Bungo Stray Dogs', 'My Hero Academia 3', 'Nanbaka',
           'Nanbaka: Season 2', 'Nanbaka: Idiots with Student Numbers!',
           'One Punch Man']
insyn_list = list(out_df[out_df['title_english'].isin(in_list)]['synopsis'])
for syn in insyn_list:
    print(syn[:300]+"...")
out_list = ['Ouran High School Host Club', 'Maid Sama!', 
            'My Teen Romantic Comedy SNAFU', 'Princess Mononoke', 'Overlord']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(out_df[out_df['title_english'] == t]['synopsis'])[0])
for syn in outsyn_list:
    print('>> \t%s'%(syn[:300]+"..."))

Centuries ago, mankind was slaughtered to near extinction by monstrous humanoid creatures called titans, forcing humans to hide in fear behind enormous concentric walls. What makes these giants truly terrifying is that their taste for human flesh is not born out of hunger but what appears to be out ...
For centuries, humanity has been hunted by giant, mysterious predators known as the Titans. Three mighty walls—Wall Maria, Rose, and Sheena—provided peace and protection for humanity for over a hundred years. That peace, however, was shattered when the Colossus Titan and Armored Titan appeared and d...
Nanba is the world's most formidable prison, built to incarcerate criminals who are too slippery to stay in ordinary confinement. The four inmates who occupy Cell 13 are particularly cunning on that behalf, having escaped every other prison with a perfect success rate. There is Juugo, a specialist i...
The seemingly ordinary and unimpressive Saitama has a rather unique hobby: being a hero.

In [272]:
# EXAMPLE 3, scored_by

in_list = ['Attack on Titan', 'Attack on Titan Season 2', 
   'Bungo Stray Dogs', 'My Hero Academia 3', 'Nanbaka', 'Nanbaka: Season 2', 'Nanbaka: Idiots with Student Numbers!',
   'One Punch Man']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['scored_by'])[0])
print(insyn_list)
out_list = ['Ouran High School Host Club', 'Maid Sama!', 
            'My Teen Romantic Comedy SNAFU', 'Princess Mononoke', 'Overlord']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['scored_by'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['scored_by']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[1038161, 439965, 190064, 262182, 57332, 26862, 5131, 806647]
>> 	354874, 	0.01635822425810041
>> 	339932, 	-0.1382430324557113
>> 	340376, 	-0.133649071943
>> 	368410, 	0.15641193934832628
>> 	344782, 	-0.08806125658487829
AVG ST.Z: 0.10654470491800325
AVG SQ ST.Z: 11391.449258105837


In [273]:
# EXAMPLE 3, popularity

in_list = ['Attack on Titan', 'Attack on Titan Season 2', 
   'Bungo Stray Dogs', 'My Hero Academia 3', 'Nanbaka', 'Nanbaka: Season 2', 'Nanbaka: Idiots with Student Numbers!',
   'One Punch Man']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['popularity'])[0])
print(insyn_list)
out_list = ['Ouran High School Host Club', 'Maid Sama!', 
            'My Teen Romantic Comedy SNAFU', 'Princess Mononoke', 'Overlord']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['popularity'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['popularity']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[2, 41, 152, 89, 658, 1534, 3340, 5]
>> 	77, 	-0.2268526811832015
>> 	76, 	-0.2272013500495734
>> 	74, 	-0.22789868778231714
>> 	79, 	-0.22615534345045776
>> 	71, 	-0.22894469438143278
AVG ST.Z: 0.22741055136939653
AVG SQ ST.Z: 652.2307035282838


In [274]:
# EXAMPLE 3, episodes

in_list = ['Attack on Titan', 'Attack on Titan Season 2', 
   'Bungo Stray Dogs', 'My Hero Academia 3', 'Nanbaka', 'Nanbaka: Season 2', 'Nanbaka: Idiots with Student Numbers!',
   'One Punch Man']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['episodes'])[0])
print(insyn_list)
out_list = ['Ouran High School Host Club', 'Maid Sama!', 
            'My Teen Romantic Comedy SNAFU', 'Princess Mononoke', 'Overlord']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['episodes'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['episodes']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[25.0, 12.0, 12.0, 25.0, 13.0, 12.0, 1.0, 12.0]
>> 	26.0, 	0.267026475725691
>> 	26.0, 	0.267026475725691
>> 	13.0, 	-0.02225220631047425
>> 	1.0, 	-0.28927868203616525
>> 	13.0, 	-0.02225220631047425
AVG ST.Z: 0.17356720922169913
AVG SQ ST.Z: 9.581231653602787


In [275]:
# EXAMPLE 3, rank

in_list = ['Attack on Titan', 'Attack on Titan Season 2', 
   'Bungo Stray Dogs', 'My Hero Academia 3', 'Nanbaka', 'Nanbaka: Season 2', 'Nanbaka: Idiots with Student Numbers!',
   'One Punch Man']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['rank'])[0])
print(insyn_list)
out_list = ['Ouran High School Host Club', 'Maid Sama!', 
            'My Teen Romantic Comedy SNAFU', 'Princess Mononoke', 'Overlord']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['rank'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['rank']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[116, 143, 858, 50, 1972, 1347, 4026, 46]
>> 	225, 	-0.3086555710105595
>> 	336, 	-0.2680982837869169
>> 	450, 	-0.226444853665338
>> 	30, 	-0.3799048593764182
>> 	473, 	-0.21804109144782646
AVG ST.Z: 0.2802289318574118
AVG SQ ST.Z: 783.9990194509174


In [276]:
# EXAMPLE 3, members

in_list = ['Attack on Titan', 'Attack on Titan Season 2', 
   'Bungo Stray Dogs', 'My Hero Academia 3', 'Nanbaka', 'Nanbaka: Season 2', 'Nanbaka: Idiots with Student Numbers!',
   'One Punch Man']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['members'])[0])
print(insyn_list)
out_list = ['Ouran High School Host Club', 'Maid Sama!', 
            'My Teen Romantic Comedy SNAFU', 'Princess Mononoke', 'Overlord']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['members'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['members']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[1500958, 781129, 401847, 535390, 143814, 57054, 15109, 1195384]
>> 	577939, 	-0.005458666449987858
>> 	579513, 	0.004123869161087997
>> 	587060, 	0.050070118371246496
>> 	568463, 	-0.06314869659646485
>> 	595128, 	0.0991882233967611
AVG ST.Z: 0.04439791479510966
AVG SQ ST.Z: 9401.446164852778


In [277]:
# EXAMPLE 3, favorites

in_list = ['Attack on Titan', 'Attack on Titan Season 2', 
   'Bungo Stray Dogs', 'My Hero Academia 3', 'Nanbaka', 'Nanbaka: Season 2', 'Nanbaka: Idiots with Student Numbers!',
   'One Punch Man']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['favorites'])[0])
print(insyn_list)
out_list = ['Ouran High School Host Club', 'Maid Sama!', 
            'My Teen Romantic Comedy SNAFU', 'Princess Mononoke', 'Overlord']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['favorites'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['favorites']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[70555, 8670, 5777, 8361, 971, 232, 30, 35969]
>> 	25017, 	1.4001484211395236
>> 	15410, 	-0.14661398065287878
>> 	15575, 	-0.1200483726279234
>> 	11828, 	-0.7233291803219102
>> 	10877, 	-0.8764436847566531
AVG ST.Z: 0.6533167278997778
AVG SQ ST.Z: 5036.444484020944


In [278]:
# EXAMPLE 3, genre_Comedy

in_list = ['Attack on Titan', 'Attack on Titan Season 2', 
   'Bungo Stray Dogs', 'My Hero Academia 3', 'Nanbaka', 'Nanbaka: Season 2', 'Nanbaka: Idiots with Student Numbers!',
   'One Punch Man']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Comedy'])[0])
print(insyn_list)
out_list = ['Ouran High School Host Club', 'Maid Sama!', 
            'My Teen Romantic Comedy SNAFU', 'Princess Mononoke', 'Overlord']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Comedy'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['genre_Comedy']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[0, 0, 1, 1, 1, 1, 1, 1]
>> 	1, 	0.5025660190455492
>> 	1, 	0.5025660190455492
>> 	1, 	0.5025660190455492
>> 	0, 	-1.5076980571366474
>> 	0, 	-1.5076980571366474
AVG ST.Z: 0.9046188342819885
AVG SQ ST.Z: 0.51234753829798
ABS MEAN DIFF: 0.15000000000000002


In [279]:
# EXAMPLE 3, genre_Action

in_list = ['Attack on Titan', 'Attack on Titan Season 2', 
   'Bungo Stray Dogs', 'My Hero Academia 3', 'Nanbaka', 'Nanbaka: Season 2', 'Nanbaka: Idiots with Student Numbers!',
   'One Punch Man']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Action'])[0])
print(insyn_list)
out_list = ['Ouran High School Host Club', 'Maid Sama!', 
            'My Teen Romantic Comedy SNAFU', 'Princess Mononoke', 'Overlord']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Action'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['genre_Action']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[1, 1, 1, 1, 1, 1, 1, 1]
>> 	0, 	-2.0470777168234116
>> 	0, 	-2.0470777168234116
>> 	0, 	-2.0470777168234116
>> 	1, 	0.0
>> 	1, 	0.0
AVG ST.Z: 1.228246630094047
AVG SQ ST.Z: 0.7745966692414834
ABS MEAN DIFF: 0.6


In [145]:
# from data_modeling EXAMPLE 4.1 (Cos, un-normalized)
in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = list(out_df[out_df['title_english'].isin(in_list)]['synopsis'])
for syn in insyn_list:
    print(syn[:300]+"...")
out_list = ['anohana: The Flower We Saw That Day', 'Parasyte -the maxim-', 
            'Elfen Lied', 'The Future Diary', 'Vampire Knight']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(out_df[out_df['title_english'] == t]['synopsis'])[0])
for syn in outsyn_list:
    print('>> \t%s'%(syn[:300]+"..."))

Moments prior to Naruto Uzumaki's birth, a huge demon known as the Kyuubi, the Nine-Tailed Fox, attacked Konohagakure, the Hidden Leaf Village, and wreaked havoc. In order to put an end to the Kyuubi's rampage, the leader of the village, the Fourth Hokage, sacrificed his life and sealed the monstrou...
The Great Kanto Desert, a sweltering wasteland of nothing but ruins and sand, is all that remains of post-apocalyptic Japan. The once fair population has been left to cling to the inhospitable dunes for survival. At least, that is the case for normal people. For those who have spent a little too lon...
In the final years of the Bakumatsu era lived a legendary assassin known as Hitokiri Battousai. Feared as a merciless killer, he was unmatched throughout the country, but mysteriously disappeared at the peak of the Japanese Revolution. It has been ten peaceful years since then, but the very mention ...
Japan, 1988. An explosion caused by a young boy with psychic powers tears through the ci

In [280]:
# EXAMPLE 4.1, popularity

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['popularity'])[0])
print(insyn_list)
out_list = ['anohana: The Flower We Saw That Day', 'Parasyte -the maxim-', 'Elfen Lied', 
            'The Future Diary', 'Vampire Knight']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['popularity'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['popularity']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[193, 1049, 10, 637, 265]
>> 	49, 	-0.13312177318078208
>> 	44, 	-0.13486511751264146
>> 	18, 	-0.14393050803831023
>> 	12, 	-0.14602252123654147
>> 	173, 	-0.08988683375066951
AVG ST.Z: 0.12956535074378894
AVG SQ ST.Z: 376.20303029082584


In [147]:
# EXAMPLE 4.1, episodes

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in _list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['episodes'])[0])
print(insyn_list)
out_list = ['anohana: The Flower We Saw That Day', 'Parasyte -the maxim-', 'Elfen Lied', 
            'The Future Diary', 'Vampire Knight']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['episodes'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['episodes']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[11.0, 24.0, 13.0, 26.0, 13.0]
>> 	11.0, 	-0.14241412038703516
>> 	24.0, 	0.1468645616491301
>> 	13.0, 	-0.09790970776608666
>> 	26.0, 	0.19136897427007857
>> 	13.0, 	-0.09790970776608666
AVG ST.Z: 0.13529341436768344
AVG SQ ST.Z: 6.2801273872430325


In [281]:
# EXAMPLE 4.1, favorites

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['favorites'])[0])
print(insyn_list)
out_list = ['anohana: The Flower We Saw That Day', 'Parasyte -the maxim-', 'Elfen Lied', 
            'The Future Diary', 'Vampire Knight']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['favorites'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['favorites']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[5845, 673, 39356, 1448, 9044]
>> 	22969, 	1.8830668990198374
>> 	18100, 	1.099139956756154
>> 	20990, 	1.5644406064053726
>> 	25002, 	2.210387390624288
>> 	8111, 	-0.5091258527061451
AVG ST.Z: 1.4532321411023592
AVG SQ ST.Z: 9760.047094148675


In [282]:
# EXAMPLE 4.1, members

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['members'])[0])
print(insyn_list)
out_list = ['anohana: The Flower We Saw That Day', 'Parasyte -the maxim-', 'Elfen Lied', 
            'The Future Diary', 'Vampire Knight']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['members'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['members']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[351163, 89817, 1091313, 148256, 296561]
>> 	734942, 	2.0670028530320677
>> 	767501, 	2.265222533439322
>> 	901294, 	3.0797563244257717
>> 	1064223, 	4.071670520472137
>> 	371097, 	-0.14809096489162654
AVG ST.Z: 2.326348639252185
AVG SQ ST.Z: 437609.31805687136


In [283]:
# EXAMPLE 4.1, genre_Comedy

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Comedy'])[0])
print(insyn_list)
out_list = ['anohana: The Flower We Saw That Day', 'Parasyte -the maxim-', 'Elfen Lied', 
            'The Future Diary', 'Vampire Knight']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Comedy'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['genre_Comedy']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[0, 1, 1, 1, 1]
>> 	0, 	-1.6082112609457573
>> 	0, 	-1.6082112609457573
>> 	0, 	-1.6082112609457573
>> 	0, 	-1.6082112609457573
>> 	0, 	-1.6082112609457573
AVG ST.Z: 1.6082112609457575
AVG SQ ST.Z: 0.8
ABS MEAN DIFF: 0.8


In [284]:
# EXAMPLE 4.1, genre_Action

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Action'])[0])
print(insyn_list)
out_list = ['anohana: The Flower We Saw That Day', 'Parasyte -the maxim-', 'Elfen Lied', 
            'The Future Diary', 'Vampire Knight']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Action'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['genre_Action']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[1, 1, 1, 1, 1]
>> 	0, 	-2.0470777168234116
>> 	1, 	0.0
>> 	1, 	0.0
>> 	1, 	0.0
>> 	0, 	-2.0470777168234116
AVG ST.Z: 0.8188310867293647
AVG SQ ST.Z: 0.6324555320336759
ABS MEAN DIFF: 0.4


In [285]:
# EXAMPLE 4.1, genre_Drama

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Drama'])[0])
print(insyn_list)
out_list = ['anohana: The Flower We Saw That Day', 'Parasyte -the maxim-', 'Elfen Lied', 
            'The Future Diary', 'Vampire Knight']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Drama'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['genre_Drama']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[0, 0, 0, 0, 0]
>> 	1, 	2.265058165791925
>> 	1, 	2.265058165791925
>> 	1, 	2.265058165791925
>> 	0, 	0.0
>> 	1, 	2.265058165791925
AVG ST.Z: 1.81204653263354
AVG SQ ST.Z: 0.8944271909999159
ABS MEAN DIFF: 0.8


In [286]:
# EXAMPLE 4.1, genre_Romance

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Romance'])[0])
print(insyn_list)
out_list = ['anohana: The Flower We Saw That Day', 'Parasyte -the maxim-', 'Elfen Lied', 
            'The Future Diary', 'Vampire Knight']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Romance'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['genre_Romance']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[0, 0, 0, 1, 1]
>> 	0, 	-0.9579256691060032
>> 	0, 	-0.9579256691060032
>> 	1, 	1.4368885036590047
>> 	0, 	-0.9579256691060032
>> 	1, 	1.4368885036590047
AVG ST.Z: 1.1495108029272036
AVG SQ ST.Z: 0.48989794855663565
ABS MEAN DIFF: 0.0


In [290]:
# EXAMPLE 4.1, genre_Horror

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Horror'])[0])
print(insyn_list)
out_list = ['anohana: The Flower We Saw That Day', 'Parasyte -the maxim-', 'Elfen Lied', 
            'The Future Diary', 'Vampire Knight']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Horror'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['genre_Horror']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[1, 0, 0, 0, 0]
>> 	0, 	-0.916742097649014
>> 	1, 	3.666968390596056
>> 	1, 	3.666968390596056
>> 	0, 	-0.916742097649014
>> 	0, 	-0.916742097649014
AVG ST.Z: 2.016832614827831
AVG SQ ST.Z: 0.5291502622129182
ABS MEAN DIFF: 0.2


In [288]:
# EXAMPLE 4.1, source_Manga

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['source_Manga'])[0])
print(insyn_list)
out_list = ['anohana: The Flower We Saw That Day', 'Parasyte -the maxim-', 'Elfen Lied', 
            'The Future Diary', 'Vampire Knight']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['source_Manga'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['source_Manga']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[1, 1, 1, 1, 1]
>> 	0, 	-2.0815569195725234
>> 	1, 	0.0
>> 	1, 	0.0
>> 	1, 	0.0
>> 	1, 	0.0
AVG ST.Z: 0.4163113839145047
AVG SQ ST.Z: 0.4472135954999579
ABS MEAN DIFF: 0.19999999999999996


In [289]:
# EXAMPLE 4.1, source_Original

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['source_Original'])[0])
print(insyn_list)
out_list = ['anohana: The Flower We Saw That Day', 'Parasyte -the maxim-', 'Elfen Lied', 
            'The Future Diary', 'Vampire Knight']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['source_Original'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['source_Original']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[0, 0, 0, 0, 0]
>> 	1, 	2.3628897854683997
>> 	0, 	0.0
>> 	0, 	0.0
>> 	0, 	0.0
>> 	0, 	0.0
AVG ST.Z: 0.47257795709367995
AVG SQ ST.Z: 0.4472135954999579
ABS MEAN DIFF: 0.2


In [149]:
# from data_modeling EXAMPLE 4.2 (Cos, normalized)
in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = list(out_df[out_df['title_english'].isin(in_list)]['synopsis'])
for syn in insyn_list:
    print(syn[:300]+"...")
out_list = ['Naruto: Shippuden', 'Bleach', 'Dragon Ball Z', 'Tokyo Ghoul √A', 'Reborn!']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(out_df[out_df['title_english'] == t]['synopsis'])[0])
for syn in outsyn_list:
    print('>> \t%s'%(syn[:300]+"..."))

Moments prior to Naruto Uzumaki's birth, a huge demon known as the Kyuubi, the Nine-Tailed Fox, attacked Konohagakure, the Hidden Leaf Village, and wreaked havoc. In order to put an end to the Kyuubi's rampage, the leader of the village, the Fourth Hokage, sacrificed his life and sealed the monstrou...
The Great Kanto Desert, a sweltering wasteland of nothing but ruins and sand, is all that remains of post-apocalyptic Japan. The once fair population has been left to cling to the inhospitable dunes for survival. At least, that is the case for normal people. For those who have spent a little too lon...
In the final years of the Bakumatsu era lived a legendary assassin known as Hitokiri Battousai. Feared as a merciless killer, he was unmatched throughout the country, but mysteriously disappeared at the peak of the Japanese Revolution. It has been ten peaceful years since then, but the very mention ...
Japan, 1988. An explosion caused by a young boy with psychic powers tears through the ci

In [291]:
# EXAMPLE 4.2, popularity

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['popularity'])[0])
print(insyn_list)
out_list = ['Naruto: Shippuden', 'Bleach', 'Dragon Ball Z', 'Tokyo Ghoul √A', 'Reborn!']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['popularity'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['popularity']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[193, 1049, 10, 637, 265]
>> 	21, 	-0.1428845014391946
>> 	22, 	-0.14253583257282273
>> 	71, 	-0.1254510581206008
>> 	42, 	-0.1355624552453852
>> 	185, 	-0.08570280735420702
AVG ST.Z: 0.12642733094644207
AVG SQ ST.Z: 367.72070923460376


In [292]:
# EXAMPLE 4.2, episodes

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['episodes'])[0])
print(insyn_list)
out_list = ['Naruto: Shippuden', 'Bleach', 'Dragon Ball Z', 'Tokyo Ghoul √A', 'Reborn!']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['episodes'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['episodes']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[1.0, 24.0, 220.0, 26.0, 94.0]
>> 	500.0, 	9.501692094572505
>> 	366.0, 	6.5198964489689555
>> 	291.0, 	4.850980975683386
>> 	12.0, 	-1.3573845849389292
>> 	203.0, 	2.8927868203616525
AVG ST.Z: 5.024548184905086
AVG SQ ST.Z: 259.3541979610124


In [293]:
# EXAMPLE 4.2, favorites

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['favorites'])[0])
print(insyn_list)
out_list = ['Naruto: Shippuden', 'Bleach', 'Dragon Ball Z', 'Tokyo Ghoul √A', 'Reborn!']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['favorites'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['favorites']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[5845, 673, 39356, 1448, 9044]
>> 	48972, 	6.069645720067807
>> 	42878, 	5.088489263679454
>> 	23468, 	1.9634077378347026
>> 	8635, 	-0.42475992176628674
>> 	14500, 	0.5195266907571274
AVG ST.Z: 2.813165866821075
AVG SQ ST.Z: 22742.684050041236


In [294]:
# EXAMPLE 4.2, members

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['members'])[0])
print(insyn_list)
out_list = ['Naruto: Shippuden', 'Bleach', 'Dragon Ball Z', 'Tokyo Ghoul √A', 'Reborn!']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['members'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['members']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[351163, 89817, 1091313, 148256, 296561]
>> 	891865, 	3.022352430984327
>> 	889504, 	3.007978627567713
>> 	591990, 	1.1967089326543574
>> 	776641, 	2.320866990545569
>> 	357485, 	-0.23096102508093055
AVG ST.Z: 1.9557736013665792
AVG SQ ST.Z: 367689.75931537716


In [295]:
# EXAMPLE 4.2, genre_Comedy

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Comedy'])[0])
print(insyn_list)
out_list = ['Naruto: Shippuden', 'Bleach', 'Dragon Ball Z', 'Tokyo Ghoul √A', 'Reborn!']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Comedy'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['genre_Comedy']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[0, 1, 1, 1, 1]
>> 	1, 	0.4020528152364392
>> 	1, 	0.4020528152364392
>> 	1, 	0.4020528152364392
>> 	0, 	-1.6082112609457573
>> 	1, 	0.4020528152364392
AVG ST.Z: 0.6432845043783029
AVG SQ ST.Z: 0.4
ABS MEAN DIFF: 0.0


In [296]:
# EXAMPLE 4.2, genre_Action

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Action'])[0])
print(insyn_list)
out_list = ['Naruto: Shippuden', 'Bleach', 'Dragon Ball Z', 'Tokyo Ghoul √A', 'Reborn!']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Action'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['genre_Action']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[1, 1, 1, 1, 1]
>> 	1, 	0.0
>> 	1, 	0.0
>> 	1, 	0.0
>> 	1, 	0.0
>> 	1, 	0.0
AVG ST.Z: 0.0
AVG SQ ST.Z: 0.0
ABS MEAN DIFF: 0


In [297]:
# EXAMPLE 4.2, genre_Romance

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Romance'])[0])
print(insyn_list)
out_list = ['Naruto: Shippuden', 'Bleach', 'Dragon Ball Z', 'Tokyo Ghoul √A', 'Reborn!']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Romance'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['genre_Romance']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[0, 0, 0, 1, 1]
>> 	0, 	-0.9579256691060032
>> 	0, 	-0.9579256691060032
>> 	0, 	-0.9579256691060032
>> 	0, 	-0.9579256691060032
>> 	0, 	-0.9579256691060032
AVG ST.Z: 0.9579256691060032
AVG SQ ST.Z: 0.4
ABS MEAN DIFF: 0.4


In [298]:
# EXAMPLE 4.2, genre_Horror

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Horror'])[0])
print(insyn_list)
out_list = ['Naruto: Shippuden', 'Bleach', 'Dragon Ball Z', 'Tokyo Ghoul √A', 'Reborn!']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Horror'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['genre_Horror']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[1, 0, 0, 0, 0]
>> 	0, 	-0.916742097649014
>> 	0, 	-0.916742097649014
>> 	0, 	-0.916742097649014
>> 	1, 	3.666968390596056
>> 	0, 	-0.916742097649014
AVG ST.Z: 1.4667873562384224
AVG SQ ST.Z: 0.4
ABS MEAN DIFF: 0.0


In [299]:
# EXAMPLE 4.2, genre_Drama

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Drama'])[0])
print(insyn_list)
out_list = ['Naruto: Shippuden', 'Bleach', 'Dragon Ball Z', 'Tokyo Ghoul √A', 'Reborn!']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Drama'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['genre_Drama']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[0, 0, 0, 0, 0]
>> 	0, 	0.0
>> 	0, 	0.0
>> 	0, 	0.0
>> 	1, 	2.265058165791925
>> 	0, 	0.0
AVG ST.Z: 0.453011633158385
AVG SQ ST.Z: 0.4472135954999579
ABS MEAN DIFF: 0.2


In [300]:
# EXAMPLE 4.2, genre_Fantasy

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Fantasy'])[0])
print(insyn_list)
out_list = ['Naruto: Shippuden', 'Bleach', 'Dragon Ball Z', 'Tokyo Ghoul √A', 'Reborn!']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Fantasy'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['genre_Fantasy']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[0, 0, 0, 1, 0]
>> 	0, 	-0.462394697808135
>> 	0, 	-0.462394697808135
>> 	1, 	1.84957879123254
>> 	0, 	-0.462394697808135
>> 	0, 	-0.462394697808135
AVG ST.Z: 0.739831516493016
AVG SQ ST.Z: 0.4000000000000001
ABS MEAN DIFF: 0.0


In [302]:
# EXAMPLE 4.2, source_Manga

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['source_Manga'])[0])
print(insyn_list)
out_list = ['Naruto: Shippuden', 'Bleach', 'Dragon Ball Z', 'Tokyo Ghoul √A', 'Reborn!']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['source_Manga'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['source_Manga']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[1, 1, 1, 1, 1]
>> 	1, 	0.0
>> 	1, 	0.0
>> 	1, 	0.0
>> 	1, 	0.0
>> 	1, 	0.0
AVG ST.Z: 0.0
AVG SQ ST.Z: 0.0
ABS MEAN DIFF: 0


In [303]:
# EXAMPLE 4.2, source_Original

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['source_Original'])[0])
print(insyn_list)
out_list = ['Naruto: Shippuden', 'Bleach', 'Dragon Ball Z', 'Tokyo Ghoul √A', 'Reborn!']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['source_Original'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['source_Original']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[0, 0, 0, 0, 0]
>> 	0, 	0.0
>> 	0, 	0.0
>> 	0, 	0.0
>> 	0, 	0.0
>> 	0, 	0.0
AVG ST.Z: 0.0
AVG SQ ST.Z: 0.0
ABS MEAN DIFF: 0


In [150]:
# from data_modeling EXAMPLE 4.4 (Eucl, normalized)
in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = list(out_df[out_df['title_english'].isin(in_list)]['synopsis'])
for syn in insyn_list:
    print(syn[:300]+"...")
out_list = ['Haikyu!! 2nd Season', 'Nisemonogatari', 'School Days', 'Wolf Children', 'Kuroko\'s Basketball 2']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(out_df[out_df['title_english'] == t]['synopsis'])[0])
for syn in outsyn_list:
    print('>> \t%s'%(syn[:300]+"..."))

Moments prior to Naruto Uzumaki's birth, a huge demon known as the Kyuubi, the Nine-Tailed Fox, attacked Konohagakure, the Hidden Leaf Village, and wreaked havoc. In order to put an end to the Kyuubi's rampage, the leader of the village, the Fourth Hokage, sacrificed his life and sealed the monstrou...
The Great Kanto Desert, a sweltering wasteland of nothing but ruins and sand, is all that remains of post-apocalyptic Japan. The once fair population has been left to cling to the inhospitable dunes for survival. At least, that is the case for normal people. For those who have spent a little too lon...
In the final years of the Bakumatsu era lived a legendary assassin known as Hitokiri Battousai. Feared as a merciless killer, he was unmatched throughout the country, but mysteriously disappeared at the peak of the Japanese Revolution. It has been ten peaceful years since then, but the very mention ...
Japan, 1988. An explosion caused by a young boy with psychic powers tears through the ci

In [304]:
# EXAMPLE 4.3, popularity

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['popularity'])[0])
print(insyn_list)
out_list = ['Haikyu!! 2nd Season', 'Nisemonogatari', 'School Days', 'Wolf Children', 'Kuroko\'s Basketball 2']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['popularity'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['popularity']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[193, 1049, 10, 637, 265]
>> 	159, 	-0.09476819787987577
>> 	151, 	-0.09755754881085078
>> 	158, 	-0.09511686674624766
>> 	149, 	-0.09825488654359453
>> 	160, 	-0.0944195290135039
AVG ST.Z: 0.09602340579881452
AVG SQ ST.Z: 275.4367440992578


In [305]:
# EXAMPLE 4.3, episodes

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['episodes'])[0])
print(insyn_list)
out_list = ['Haikyu!! 2nd Season', 'Nisemonogatari', 'School Days', 'Wolf Children', 'Kuroko\'s Basketball 2']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['episodes'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['episodes']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[1.0, 24.0, 220.0, 26.0, 94.0]
>> 	25.0, 	-1.068105902902764
>> 	11.0, 	-1.3796367912494034
>> 	12.0, 	-1.3573845849389292
>> 	1.0, 	-1.602158854354146
>> 	25.0, 	-1.068105902902764
AVG ST.Z: 1.2950784072696013
AVG SQ ST.Z: 58.91858789889656


In [306]:
# EXAMPLE 4.3, favorites

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['favorites'])[0])
print(insyn_list)
out_list = ['Haikyu!! 2nd Season', 'Nisemonogatari', 'School Days', 'Wolf Children', 'Kuroko\'s Basketball 2']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['favorites'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['favorites']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[5845, 673, 39356, 1448, 9044]
>> 	7426, 	-0.6194133769309599
>> 	2839, 	-1.3579372800247196
>> 	2550, 	-1.4044673449896414
>> 	7779, 	-0.5625790761260553
>> 	3717, 	-1.216576044594957
AVG ST.Z: 1.0321946245332667
AVG SQ ST.Z: 6802.006877385527


In [307]:
# EXAMPLE 4.3, members

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['members'])[0])
print(insyn_list)
out_list = ['Haikyu!! 2nd Season', 'Nisemonogatari', 'School Days', 'Wolf Children', 'Kuroko\'s Basketball 2']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['members'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['members']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[351163, 89817, 1091313, 148256, 296561]
>> 	392000, 	-0.020833187332338993
>> 	401291, 	0.03573056003901156
>> 	390373, 	-0.030738387738451074
>> 	404379, 	0.054530350361122255
>> 	390398, 	-0.030586187363433985
AVG ST.Z: 0.03448373456687158
AVG SQ ST.Z: 5951.740434528374


In [308]:
# EXAMPLE 4.2, genre_Comedy

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Comedy'])[0])
print(insyn_list)
out_list = ['Haikyu!! 2nd Season', 'Nisemonogatari', 'School Days', 'Wolf Children', 'Kuroko\'s Basketball 2']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Comedy'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['genre_Comedy']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[0, 1, 1, 1, 1]
>> 	1, 	0.4020528152364392
>> 	1, 	0.4020528152364392
>> 	0, 	-1.6082112609457573
>> 	0, 	-1.6082112609457573
>> 	1, 	0.4020528152364392
AVG ST.Z: 0.8845161935201664
AVG SQ ST.Z: 0.5291502622129182
ABS MEAN DIFF: 0.20000000000000007


In [309]:
# EXAMPLE 4.2, genre_Action

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Action'])[0])
print(insyn_list)
out_list = ['Haikyu!! 2nd Season', 'Nisemonogatari', 'School Days', 'Wolf Children', 'Kuroko\'s Basketball 2']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Action'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['genre_Action']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[1, 1, 1, 1, 1]
>> 	0, 	-2.0470777168234116
>> 	0, 	-2.0470777168234116
>> 	0, 	-2.0470777168234116
>> 	0, 	-2.0470777168234116
>> 	0, 	-2.0470777168234116
AVG ST.Z: 2.0470777168234116
AVG SQ ST.Z: 1.0
ABS MEAN DIFF: 1


In [310]:
# EXAMPLE 4.2, genre_Drama

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Drama'])[0])
print(insyn_list)
out_list = ['Haikyu!! 2nd Season', 'Nisemonogatari', 'School Days', 'Wolf Children', 'Kuroko\'s Basketball 2']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Drama'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['genre_Drama']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[0, 0, 0, 0, 0]
>> 	1, 	2.265058165791925
>> 	0, 	0.0
>> 	1, 	2.265058165791925
>> 	0, 	0.0
>> 	0, 	0.0
AVG ST.Z: 0.90602326631677
AVG SQ ST.Z: 0.6324555320336759
ABS MEAN DIFF: 0.4


In [311]:
# EXAMPLE 4.2, genre_Fantasy

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Fantasy'])[0])
print(insyn_list)
out_list = ['Haikyu!! 2nd Season', 'Nisemonogatari', 'School Days', 'Wolf Children', 'Kuroko\'s Basketball 2']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Fantasy'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['genre_Fantasy']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[0, 0, 0, 1, 0]
>> 	0, 	-0.462394697808135
>> 	0, 	-0.462394697808135
>> 	0, 	-0.462394697808135
>> 	1, 	1.84957879123254
>> 	0, 	-0.462394697808135
AVG ST.Z: 0.739831516493016
AVG SQ ST.Z: 0.4
ABS MEAN DIFF: 0.0


In [313]:
# EXAMPLE 4.2, genre_Romance

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Romance'])[0])
print(insyn_list)
out_list = ['Haikyu!! 2nd Season', 'Nisemonogatari', 'School Days', 'Wolf Children', 'Kuroko\'s Basketball 2']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Romance'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['genre_Romance']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[0, 0, 0, 1, 1]
>> 	0, 	-0.9579256691060032
>> 	0, 	-0.9579256691060032
>> 	1, 	1.4368885036590047
>> 	0, 	-0.9579256691060032
>> 	0, 	-0.9579256691060032
AVG ST.Z: 1.0537182360166033
AVG SQ ST.Z: 0.4472135954999579
ABS MEAN DIFF: 0.2


In [314]:
# EXAMPLE 4.2, genre_Horror

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Horror'])[0])
print(insyn_list)

out_list = ['Haikyu!! 2nd Season', 'Nisemonogatari', 'School Days', 'Wolf Children', 'Kuroko\'s Basketball 2']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Horror'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['genre_Horror']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[1, 0, 0, 0, 0]
>> 	0, 	-0.916742097649014
>> 	0, 	-0.916742097649014
>> 	0, 	-0.916742097649014
>> 	0, 	-0.916742097649014
>> 	0, 	-0.916742097649014
AVG ST.Z: 0.916742097649014
AVG SQ ST.Z: 0.2
ABS MEAN DIFF: 0.2


In [315]:
# EXAMPLE 4.2, source_Manga

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['source_Manga'])[0])
print(insyn_list)
out_list = ['Haikyu!! 2nd Season', 'Nisemonogatari', 'School Days', 'Wolf Children', 'Kuroko\'s Basketball 2']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['source_Manga'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['source_Manga']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[1, 1, 1, 1, 1]
>> 	1, 	0.0
>> 	0, 	-2.0815569195725234
>> 	0, 	-2.0815569195725234
>> 	0, 	-2.0815569195725234
>> 	1, 	0.0
AVG ST.Z: 1.248934151743514
AVG SQ ST.Z: 0.7745966692414834
ABS MEAN DIFF: 0.6


In [316]:
# EXAMPLE 4.2, source_Original

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['source_Original'])[0])
print(insyn_list)
out_list = ['Haikyu!! 2nd Season', 'Nisemonogatari', 'School Days', 'Wolf Children', 'Kuroko\'s Basketball 2']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['source_Original'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['source_Original']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[0, 0, 0, 0, 0]
>> 	0, 	0.0
>> 	0, 	0.0
>> 	0, 	0.0
>> 	1, 	2.3628897854683997
>> 	0, 	0.0
AVG ST.Z: 0.47257795709367995
AVG SQ ST.Z: 0.4472135954999579
ABS MEAN DIFF: 0.2


In [151]:
# from data_modeling EXAMPLE 4.3 (Eucl, un-normalized)
in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = list(out_df[out_df['title_english'].isin(in_list)]['synopsis'])
for syn in insyn_list:
    print(syn[:300]+"...")
out_list = ['JoJo\'s Bizarre Adventure: Stardust Crusaders', 'Drifters', 
            'JoJo\'s Bizarre Adventure', 'Evangelion: 3.0 You Can (Not) Redo', 
            'Re:CREATORS']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(out_df[out_df['title_english'] == t]['synopsis'])[0])
for syn in outsyn_list:
    print('>> \t%s'%(syn[:300]+"..."))

Moments prior to Naruto Uzumaki's birth, a huge demon known as the Kyuubi, the Nine-Tailed Fox, attacked Konohagakure, the Hidden Leaf Village, and wreaked havoc. In order to put an end to the Kyuubi's rampage, the leader of the village, the Fourth Hokage, sacrificed his life and sealed the monstrou...
The Great Kanto Desert, a sweltering wasteland of nothing but ruins and sand, is all that remains of post-apocalyptic Japan. The once fair population has been left to cling to the inhospitable dunes for survival. At least, that is the case for normal people. For those who have spent a little too lon...
In the final years of the Bakumatsu era lived a legendary assassin known as Hitokiri Battousai. Feared as a merciless killer, he was unmatched throughout the country, but mysteriously disappeared at the peak of the Japanese Revolution. It has been ten peaceful years since then, but the very mention ...
Japan, 1988. An explosion caused by a young boy with psychic powers tears through the ci

In [317]:
# EXAMPLE 4.3, popularity

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['popularity'])[0])
print(insyn_list)
out_list = ['JoJo\'s Bizarre Adventure: Stardust Crusaders', 'Drifters', 
            'JoJo\'s Bizarre Adventure', 'Evangelion: 3.0 You Can (Not) Redo', 
            'Re:CREATORS']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['popularity'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['popularity']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[193, 1049, 10, 637, 265]
>> 	289, 	-0.04944124525153196
>> 	294, 	-0.047697900919672576
>> 	2139, 	0.5955961575364378
>> 	465, 	0.011924475229918139
>> 	333, 	-0.03409981513116943
AVG ST.Z: 0.147751918813746
AVG SQ ST.Z: 770.3899272446389


In [318]:
# EXAMPLE 4.3, episodes

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['episodes'])[0])
print(insyn_list)
out_list = ['JoJo\'s Bizarre Adventure: Stardust Crusaders', 'Drifters', 
            'JoJo\'s Bizarre Adventure', 'Evangelion: 3.0 You Can (Not) Redo', 
            'Re:CREATORS']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['episodes'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['episodes']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[1.0, 24.0, 220.0, 26.0, 94.0]
>> 	24.0, 	-1.0903581092132382
>> 	12.0, 	-1.3573845849389292
>> 	7.0, 	-1.4686456164913004
>> 	1.0, 	-1.602158854354146
>> 	22.0, 	-1.1348625218341868
AVG ST.Z: 1.33068193736636
AVG SQ ST.Z: 60.436743790512075


In [319]:
# EXAMPLE 4.3, favorites

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['favorites'])[0])
print(insyn_list)
out_list = ['JoJo\'s Bizarre Adventure: Stardust Crusaders', 'Drifters', 
            'JoJo\'s Bizarre Adventure', 'Evangelion: 3.0 You Can (Not) Redo', 
            'Re:CREATORS']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['favorites'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['favorites']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[5845, 673, 39356, 1448, 9044]
>> 	6335, 	-0.7950683972656649
>> 	2120, 	-1.473698929539525
>> 	166, 	-1.788300130028997
>> 	983, 	-1.6567601193842176
>> 	2044, 	-1.4859352095995046
AVG ST.Z: 1.4399525571635816
AVG SQ ST.Z: 9193.50999346822


In [320]:
# EXAMPLE 4.3, members

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['members'])[0])
print(insyn_list)
out_list = ['JoJo\'s Bizarre Adventure: Stardust Crusaders', 'Drifters', 
            'JoJo\'s Bizarre Adventure', 'Evangelion: 3.0 You Can (Not) Redo', 
            'Re:CREATORS']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['members'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['members']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[351163, 89817, 1091313, 148256, 296561]
>> 	281554, 	-0.6932300920978306
>> 	275539, 	-0.729849502326942
>> 	34055, 	-2.2000077167520002
>> 	193319, 	-1.2304060956831406
>> 	248516, 	-0.8943659316904127
AVG ST.Z: 1.149571867710065
AVG SQ ST.Z: 209929.0162159581


In [321]:
# EXAMPLE 4.2, genre_Comedy

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Comedy'])[0])
print(insyn_list)
out_list = ['JoJo\'s Bizarre Adventure: Stardust Crusaders', 'Drifters', 
            'JoJo\'s Bizarre Adventure', 'Evangelion: 3.0 You Can (Not) Redo', 
            'Re:CREATORS']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Comedy'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['genre_Comedy']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[0, 1, 1, 1, 1]
>> 	0, 	-1.6082112609457573
>> 	1, 	0.4020528152364392
>> 	0, 	-1.6082112609457573
>> 	0, 	-1.6082112609457573
>> 	0, 	-1.6082112609457573
AVG ST.Z: 1.3669795718038935
AVG SQ ST.Z: 0.7211102550927979
ABS MEAN DIFF: 0.6000000000000001


In [322]:
# EXAMPLE 4.2, genre_Action

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Action'])[0])
print(insyn_list)
out_list = ['JoJo\'s Bizarre Adventure: Stardust Crusaders', 'Drifters', 
            'JoJo\'s Bizarre Adventure', 'Evangelion: 3.0 You Can (Not) Redo', 
            'Re:CREATORS']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Action'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['genre_Action']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[1, 1, 1, 1, 1]
>> 	1, 	0.0
>> 	1, 	0.0
>> 	0, 	-2.0470777168234116
>> 	1, 	0.0
>> 	1, 	0.0
AVG ST.Z: 0.40941554336468233
AVG SQ ST.Z: 0.4472135954999579
ABS MEAN DIFF: 0.19999999999999996


In [323]:
# EXAMPLE 4.2, genre_Drama

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Drama'])[0])
print(insyn_list)
out_list = ['JoJo\'s Bizarre Adventure: Stardust Crusaders', 'Drifters', 
            'JoJo\'s Bizarre Adventure', 'Evangelion: 3.0 You Can (Not) Redo', 
            'Re:CREATORS']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Drama'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['genre_Drama']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[0, 0, 0, 0, 0]
>> 	1, 	2.265058165791925
>> 	0, 	0.0
>> 	1, 	2.265058165791925
>> 	0, 	0.0
>> 	0, 	0.0
AVG ST.Z: 0.90602326631677
AVG SQ ST.Z: 0.6324555320336759
ABS MEAN DIFF: 0.4


In [324]:
# EXAMPLE 4.2, genre_Fantasy

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Fantasy'])[0])
print(insyn_list)
out_list = ['JoJo\'s Bizarre Adventure: Stardust Crusaders', 'Drifters', 
            'JoJo\'s Bizarre Adventure', 'Evangelion: 3.0 You Can (Not) Redo', 
            'Re:CREATORS']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Fantasy'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['genre_Fantasy']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[0, 0, 0, 1, 0]
>> 	0, 	-0.462394697808135
>> 	1, 	1.84957879123254
>> 	1, 	1.84957879123254
>> 	0, 	-0.462394697808135
>> 	1, 	1.84957879123254
AVG ST.Z: 1.2947051538627778
AVG SQ ST.Z: 0.6324555320336759
ABS MEAN DIFF: 0.39999999999999997


In [328]:
# EXAMPLE 4.2, genre_Romance

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Romance'])[0])
print(insyn_list)
out_list = ['JoJo\'s Bizarre Adventure: Stardust Crusaders', 'Drifters', 
            'JoJo\'s Bizarre Adventure', 'Evangelion: 3.0 You Can (Not) Redo', 
            'Re:CREATORS']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Romance'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['genre_Romance']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[0, 0, 0, 1, 1]
>> 	0, 	-0.9579256691060032
>> 	0, 	-0.9579256691060032
>> 	0, 	-0.9579256691060032
>> 	0, 	-0.9579256691060032
>> 	0, 	-0.9579256691060032
AVG ST.Z: 0.9579256691060032
AVG SQ ST.Z: 0.4
ABS MEAN DIFF: 0.4


In [329]:
# EXAMPLE 4.2, genre_Horror

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Horror'])[0])
print(insyn_list)
out_list = ['JoJo\'s Bizarre Adventure: Stardust Crusaders', 'Drifters', 
            'JoJo\'s Bizarre Adventure', 'Evangelion: 3.0 You Can (Not) Redo', 
            'Re:CREATORS']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['genre_Horror'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['genre_Horror']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[1, 0, 0, 0, 0]
>> 	0, 	-0.916742097649014
>> 	0, 	-0.916742097649014
>> 	1, 	3.666968390596056
>> 	0, 	-0.916742097649014
>> 	0, 	-0.916742097649014
AVG ST.Z: 1.4667873562384226
AVG SQ ST.Z: 0.4000000000000001
ABS MEAN DIFF: 0.0


In [325]:
# EXAMPLE 4.2, source_Manga

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['source_Manga'])[0])
print(insyn_list)
out_list = ['JoJo\'s Bizarre Adventure: Stardust Crusaders', 'Drifters', 
            'JoJo\'s Bizarre Adventure', 'Evangelion: 3.0 You Can (Not) Redo', 
            'Re:CREATORS']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['source_Manga'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['source_Manga']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[1, 1, 1, 1, 1]
>> 	1, 	0.0
>> 	1, 	0.0
>> 	1, 	0.0
>> 	0, 	-2.0815569195725234
>> 	0, 	-2.0815569195725234
AVG ST.Z: 0.8326227678290093
AVG SQ ST.Z: 0.6324555320336759
ABS MEAN DIFF: 0.4


In [327]:
# EXAMPLE 4.2, source_Original

in_list = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
insyn_list = []
for t in in_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['source_Original'])[0])
print(insyn_list)
out_list = ['JoJo\'s Bizarre Adventure: Stardust Crusaders', 'Drifters', 
            'JoJo\'s Bizarre Adventure', 'Evangelion: 3.0 You Can (Not) Redo', 
            'Re:CREATORS']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['source_Original'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), f_stdv['source_Original']))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))
print("ABS MEAN DIFF: %s"%(abs(mean(outsyn_list)-mean(insyn_list))))

[0, 0, 0, 0, 0]
>> 	0, 	0.0
>> 	0, 	0.0
>> 	0, 	0.0
>> 	1, 	2.3628897854683997
>> 	1, 	2.3628897854683997
AVG ST.Z: 0.9451559141873599
AVG SQ ST.Z: 0.6324555320336759
ABS MEAN DIFF: 0.4
